<a href="https://colab.research.google.com/github/RuiCal/colab_notebooks/blob/main/salesforce_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SalesForce API Data Connector

## Classes

In [1]:
%%time
%%capture
# Install missing libraries in Colab
!pip install simple-salesforce
!pip install sidecar
import os
# This is a bad practice; we need to use a SalesForce API client Token instead;
os.environ['SFDC_SID'] = '00D50000000JKHo!AQkAQP6HDGtfu2U_Uiwnab8UGFlAy8kBhT7JwDVbs5nE0UrTPGx7kB_EbxvC6nojIJ6cpGhcCOriv57ej0zsyNnll5XERjlm'
os.environ['SLACK_BOT_TOKEN'] = ''

CPU times: user 183 ms, sys: 31.3 ms, total: 214 ms
Wall time: 23.8 s


### SalesForce Manager

In [2]:
''' Import Libraries '''
# -*- Data Manipulation  -*-
import pandas as pd 
import numpy as np
import dask.dataframe as dd

#  -*- I/O, Error Handling -*-
import os, sys, json, traceback
from os import path
import warnings, time
from pathlib import Path 
from datetime import datetime, date 
from collections import OrderedDict
from google.colab import drive

# -* To open separate Tabs in Jupyter Notebook -*-
from sidecar import Sidecar

# -*- Set visual styler options for Pandas DataFrame outputs -*- 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
cm = sns.color_palette('viridis', as_cmap=True)

# -*- Simple-SalesForce API -*-
from simple_salesforce import Salesforce 
from simple_salesforce.exceptions import SalesforceExpiredSession

# -*- Set Notebook Options -*- 
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_info_columns', 500)  
pd.set_option('display.max_rows', None) 
from IPython.display import HTML, display
import ipywidgets as widgets

# -*- Fast selecting and setting with Pandas DataFrame Objects using loc or iloc -*-  
idx = pd.IndexSlice 
All = slice(None)

# -*- Constants
GOOGLE_PATH = '/content/gdrive/'
SESSION_URL = 'https://udemy.my.salesforce.com/secur/frontdoor.jsp?'


class sf_Manager():
    
    def __init__(self, **kwargs):
        
        '''
            Description:
                Initialize SalesForce Manager Class with kwargs parameters, plus session parameters for SalesForce Authentication Session

                Kwargs Required parameters:
                    * [acct_labels, opp_labels]: SalesForce SOQL Account and Opportunity fields to query

                SalesForce Authentication:
                    *username: the Salesforce username to use for authentication;
                    *password: the Salesforce password for the username;
                    *token: the Salesforce security API token for the username (retrieved from Salesforce UI) # does not work; disabled in our SalesForce instance;
                    *instance: Domain of your Salesforce instance, e.g.`domain1.salesforce.com`;
                    *sid: Session ID for Auth; retrieve SID from set-cookie headers in https://udemy.my.salesforce.com/secur/frontdoor.jsp?

                Data Manipulation:
                    *inr_to_usd: to convert Opportunities in Indian Currenty to USD.
                    *account_fields: not used with any method, but handy to print all fields for the SalesForce Account object
                    *opportunity_fields: not used with any method, but handy to print all fields for the SalesForce Opportunity object
                    *case_fields: not used with any method, but handy to print all fields for the SalesForce Case object
                    *SSEIDs: SalesForce User account IDs for the S&SE Team; used with Cases DataFrame object
                    *years: default Years to retrieve Opportunity Data from SOQL Query
        '''
        
        super(sf_Manager, self).__init__()
        self.kwargs = kwargs
        
        # Mount Google Drive
        self._mount_google_drive()

        # Set Google Colab to Wrap Text on Outputs
        def set_css():
          display(HTML('''
          <style>
            pre {
                white-space: pre-wrap;
            }
          </style>
          '''))
        get_ipython().events.register('pre_run_cell', set_css)

        # SalesForce User Creds: these are in set in your SalesForce User Profile settings
        self.instance = 'udemy.my.salesforce.com'
        self.sid = os.environ['SFDC_SID']

        # Get Session
        self.sf = self.conn()
        
        # Get S&SE Assignments Map
        self.sse_salesforce_opp_assignments_path = GOOGLE_PATH + 'Shareddrives/UB/Sales/Sales & Solution Engineers/Sales/SalesForce/sse_salesforce_opp_assignments_2023v1.json'
        self.sse_assignments = self._open_json_file(path_or_buf=self.sse_salesforce_opp_assignments_path)
        
        # Used for Currency conversion for India Opp ARR
        self.inr_to_usd = .014
         
        # These are the available Salesforce Fields for each Entity Type (e.g., Account, Opportunity, Case)
        # More info here: https://developer.salesforce.com/docs/atlas.en-us.234.0.sfFieldRef.meta/sfFieldRef/salesforce_field_reference.htm
        self.user_fields = ",".join([field['name'] for field in self.sf.User.describe()['fields']])
        self.account_fields = ",".join([field['name'] for field in self.sf.Account.describe()['fields']])
        self.opportunity_fields = ",".join([field['name'] for field in self.sf.Opportunity.describe()['fields']])
        self.case_fields = ",".join([field['name'] for field in self.sf.Case.describe()['fields']])
        self.contact_fields = ",".join([field['name'] for field in self.sf.Contact.describe()['fields']])
        self.ironclad_workflow_fields = ",".join([field['name'] for field in self.sf.ironclad__Ironclad_Workflow__c.describe()['fields']])
        self.ironclad__Ironclad_Approval_fields = ",".join([field['name'] for field in self.sf.ironclad__Ironclad_Approval__c.describe()['fields']])
        self.OpportunityChangeEvent_fields = ",".join([field['name'] for field in self.sf.OpportunityChangeEvent.describe()['fields']])
        self.OpportunityFieldHistory_fields = ",".join([field['name'] for field in self.sf.OpportunityFieldHistory.describe()['fields']])
        self.OpportunityStage_fields = ",".join([field['name'] for field in self.sf.OpportunityStage.describe()['fields']])
        self.OpportunityHistory_fields = ",".join([field['name'] for field in self.sf.OpportunityHistory.describe()['fields']])
        self.OpportunityLineItem_fields = ",".join([field['name'] for field in self.sf.OpportunityLineItem.describe()['fields']])
        self.Product2_fields =  ",".join([field['name'] for field in self.sf.Product2.describe()['fields']])
        self.Task_fields =  ",".join([field['name'] for field in self.sf.Task.describe()['fields']])
        
        # Get SalesForce User IDs for the S&SE Team; 
        # Use these IDs to filter SalesForce SOQL Query for S&SE assigned Cases
        soql_query = "SELECT Id, Name FROM User WHERE email IN " + \
                     "('daniel.mcateer@udemy.com', 'garry.hickey@udemy.com', 'rui.calheiros@udemy.com', 'seth.klein@udemy.com', 'steven.reynolds@udemy.com', " + \
                     "'vamsi.pandari@udemy.com', 'ayse.ciftci@udemy.com', 'david.vaughan@udemy.com', 'ryan.petigura@udemy.com', 'joao.carlos@udemy.com', " + \
                     "'erin.ohern@udemy.com', 'paul.sleeman@udemy.com', 'sawandi.cassell@udemy.com', 'jenica.draney@udemy.com', 'christopher.mcdonald@udemy.com', " \
                     "'dan.fortner@udemy.com', 'kei.pan@udemy.com', 'evelyn.pan@udemy.com', 'william.holton@udemy.com', 'james.caudery@udemy.com', 'kuldeep.harsh@udemy.com', " \
                     "'lynsie.zellmer@udemy.com', 'sarah.sparks@udemy.com')" 
        
        self.SSE_IDs = self.dict_to_df(soql_result=self.query(query=soql_query))
        self.SSE_IDs.loc[len(self.SSE_IDs.index)] = ['00G38000002vdbeEAA', 'UFB SE Request',]
        self.SSE_IDs.loc[len(self.SSE_IDs.index)] = ['00G38000002bt1KEAQ', 'UFB Request for Proposal']
        # sys.stdout.write('--------------------\n{0}\n--------------------'.format(self.SSE_IDs))
        self.SSEIDs = list(self.SSE_IDs.Id)
        self.SSEIDs = ", ".join(["'{}'".format(element) for element in self.SSEIDs])
        
        # Define SOQL Syntax fields to query Accounts
        acct_labels = ['o.Account.{}'.format(field) for field in self.kwargs.get('acct_labels').split(',')]
        acct_labels = ",".join([field for field in acct_labels]) 
        acct_labels =  "".join(acct_labels.split())
        acct_labels = acct_labels.replace(",", ", ")          
        self.acct_labels = acct_labels
        
        # Define SOQL Syntax fields to query Opportunities
        opp_labels = ['o.{}'.format(field) for field in self.kwargs.get('opp_labels').split(',')]
        opp_labels = ",".join([field for field in opp_labels]) 
        opp_labels = "".join(opp_labels.split())
        opp_labels = opp_labels.replace(",", ", ")
        self.opp_labels = opp_labels
        
        # Default years to filter SalesForce Opportunities
        self.years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
        
    def conn(self):
        '''
        Connects to SalesForce Web Services using simple_salesforce library and returns session 
        
        Parameters:
            *username (string): Salesforce username to use for authentication
            *password (sting):  Salesforce password for the username
            *token(string): Salesforce security API token for the username (retrieved from Salesforce UI)
            *instance (string): Domain of your Salesforce instance, e.g.`domain1.salesforce.com`
        
        Returns: 
            SalesForce API Session
        '''
        try:
            '''
            This auth method now fails because SF Admin team disabled user-level authentication
            self.sf = Salesforce(username=self.username,
                                 password=self.password, 
                                 security_token=self.token, 
                                 instance=self.instance
                                )
            '''
            session = Salesforce(instance=self.instance, session_id=self.sid, version='52.0')
            sys.stdout.write('\n{0}{1}: Connected to {2}{3}\n'.format('\x1b[32m', datetime.now(), session.base_url, '\x1b[32m'))
            return session
    
        except SalesforceExpiredSession as error:
            self.sf = Salesforce(instance=self.instance, session_id=self.sid)
            return self.sf
            sys.stdout.write('\n {0} *** Error in <sf_Manager.conn()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc())) 
    
    def query(self, query):
        
        '''
        Returns SalesForce OrderedDict Object Query
        
        Parameters:
            query (string): SalesForce Query (SOQL)
        
        Returns:
            result (OrderedDict): SalesForce OrderedDict Object
        '''
        
        try:
            self.conn()
            sys.stdout.write('\n{0}Querying SOQL Data{1}'.format('\x1b[32m', '\x1b[32m'))
            sys.stdout.write('\nSOQL Query: \n{0}{1}{2}\n'.format('\x1b[6;30;43m', query, '\x1b[0m'))
            result = self.sf.query_all(query)
            return result
        except SalesforceExpiredSession as error:
            sys.stdout.write('\n {0} *** Error in <sf_Manager.query()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))   
            return dict()
    
    def dict_to_df(self, soql_result):
        
        '''
        Returns Pandas DataFrame object of a SalesForce OrderedDict query
        
        Parameters:
            soql_result: (OrderedDict): SalesForce API OrderedDict Object
        
        Returns:
            df (Pandas DataFrame): Pandas DataFrame object of soql_result
        '''       
        try:
            items = {val: dict(soql_result['records'][val]) for val in range(soql_result['totalSize'])}
            df = pd.DataFrame.from_dict(items, orient='index').drop(['attributes'], axis=1)
            return df
        
        except Exception as error:
            sys.stdout.write('\n {0} *** Error in <sf_Manager.dict_to_df()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))            
    
    def get_salesforce_object_entities(self):
        
        '''
        Description: Function that returns metadata for all SalesForce API Objects
        Parameters:     
             None
        Returns:
            sf_objects (OrderedDict) = SalesForce Data Model Objects (e.g., Account, Opportunity, Case)
        '''
        try:
            sf = self.conn()
            sf_objects = []
            for sfobject in sf.describe()['sobjects']:
                sf_objects.append(sfobject['name']) 
            return sf_objects
        
        except Exception as error:
            sys.stdout.write('\n {0} *** Error in <sf_Manager.get_salesforce_object_entities()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc())) 

    def get_users(self):
        
        '''
        Queries SalesForce User data and Returns Pandas DataFrame as result
           
           Parameters:
                None
            
            Returns:
                users (Pandas DataFrame): Pandas DataFrame Object of SalesForce Users data
        '''
        try:
            sys.stdout.write('\n********  {0}Getting SalesForce Users{1}  ********\n'.format('\x1b[6;30;43m', '\x1b[0m'))
            soql_query = "SELECT Id, Name, Title, Department, Manager_for_Reports__c, Email FROM User"
            soql_result = self.query(query=soql_query)
            users = self.dict_to_df(soql_result=soql_result)
            users.columns = 'user_' + users.columns
            print(users.info())
            return users
        except Exception as error:
            sys.stdout.write('\n {0} *** Error in <sf_Manager.get_users()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc())) 
            return pd.DataFrame()
        
    def get_accounts(self):
        
        '''
        Queries SalesForce Accounts Data and Returns Pandas DataFrame as result
        
        Parameters: 
                None
        
        Returns: 
                accounts (Pandas DataFrame): Pandas DataFrame Object of SalesForce Accounts OrderedDict Account data
        '''        
        try:
            data = pd.DataFrame()
            sys.stdout.write('\n********  {0}Getting Accounts{1}  ********\n'.format('\x1b[6;30;43m', '\x1b[0m'))
            soql_query = "SELECT " + self.kwargs.get('account_labels') + " FROM Account"
            data = self.dict_to_df(soql_result=self.query(query=soql_query))
            data.columns = 'acct_' + data.columns
            data.sort_index(axis=1, ascending=True, inplace=True)
            print(data.info())
            print('\nNo. of Accounts: {}'.format(len(data)))
            return data
        except Exception as error:
            sys.stdout.write('\n {0} *** Error in <sf_Manager.get_accounts()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc())) 
            return pd.DataFrame()

    def get_accounts_opportunities(self, years=[pd.Timestamp(datetime.now()).year]):
        
        '''
        Queries SalesForce Accounts & Opportunities in single call and converts results to Pandas DataFrame objects
        
        Parameters:
            years (list(int)): Years to filter SalesForce Account and Opportunity Query
       
        Returns:
            Accounts (Pandas DataFrame): Pandas DataFrame object of SalesForce Accounts data (parent records)
            Opportunities (Pandas DataFrame): Pandas DataFrame object with Opportunity Data (child records)
        ''' 
        
        try:
            
            # (1) Get SalesForce Opportunities data; (2) Convert OrderedDict result to Pandas DataFrame
            opportunities = pd.DataFrame()
            sys.stdout.write('\n\n******** {0}Getting Accounts & Opps Data for {1}{2} ********\n\n'.format('\x1b[6;30;43m', years, '\x1b[0m'))
            for year in years:
                soql_query = "SELECT " + str(self.opp_labels) + ", " + str(self.acct_labels) + " FROM Opportunity o WHERE o.FiscalYear IN (" + str(year) + ")"
                soql_results = self.dict_to_df(soql_result=sf.query(query=soql_query))
                sys.stdout.write('\n{0}{1} No. of Opportunities: {2} {3}\n'.format('\x1b[6;30;46m',  year, len(soql_results), '\x1b[0m'))
                opportunities = pd.concat([opportunities, soql_results], ignore_index=True, sort=False)
       
            # Clean up data
            opportunities.columns = 'opp_' + opportunities.columns # add prefix to make it easier to distinguish data features from each object
            opportunities.drop_duplicates(subset=['opp_Id'], inplace=True)  # drop any duplicate Opps
            opportunities.dropna(subset=['opp_AccountId'],inplace=True) # Some Opportunities aren't tagged with an Account, drop these records
            
            # Extract the nested Accounts Data from query results
            sys.stdout.write('\n{}>> Parsing Accounts and Opportunities Records. . . {}\n'.format('\x1b[6;30;46m', '\x1b[0m'))
            accounts = {}
            accounts['records'] = opportunities['opp_Account'].to_list()
            accounts['totalSize'] = len(opportunities)
            accounts['done'] = True
            
            # Convert Accounts Dict to Pandas DataFrame
            accounts = self.dict_to_df(soql_result=accounts)    
            accounts.columns = 'acct_' + accounts.columns
            accounts.drop_duplicates(subset=['acct_Id'], inplace=True)
            
            # Print Results
            sys.stdout.write('\n\n -*- {0} Account Records {1} -*-\n\n'.format('\x1b[6;30;46m','\x1b[0m'))
            print(accounts.info())
            sys.stdout.write('\n\n -*- {0} Opportunity Records {1} -*-\n\n'.format('\x1b[6;30;46m', '\x1b[0m'))
            print(opportunities.info())
            return accounts, opportunities
        
        except Exception as error:   
            sys.stdout.write('\n {0} *** Error: <sf_Manager.get_accounts_opportunities()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc())) 
            return pd.DataFrame()

    def get_opportunity_field_history(self, fields=['StageName']):
        '''
        Queries SalesForce OpportunityFieldHistory Object and returns results as Pandas DataFrame object
        
        Parameters:
            fields (list): List of Fields to return changes for in the Opportunity
       
        Returns:
            Data (Pandas DataFrame): OpportunityFieldHistory
        ''' 
        try:
            data = pd.DataFrame()
            sys.stdout.write('\n\n******** {0} Getting Opportunity Field History Data {1}  ******** \n'.format('\x1b[6;30;43m', '\x1b[0m'))
            fields = ", ".join(["'{}'".format(element) for element in fields])
            soql_query = "SELECT " + self.OpportunityFieldHistory_fields + " FROM OpportunityFieldHistory WHERE Field IN (" + fields + ") AND OpportunityId='0061T000015aml1QAA'"
            data = sf.dict_to_df(soql_result=sf.query(query=soql_query))
            data.columns = 'oppfieldhist_' + data.columns
            
            # Convert to DateTime
            date_labels = ['oppfieldhist_CreatedDate']
            data[date_labels] = data[date_labels].apply(pd.to_datetime)
            data['oppfieldhist_CreatedDate_'] = data['oppfieldhist_CreatedDate']
            data.sort_values(by=['oppfieldhist_CreatedDate'], ascending=[True], inplace=True)
            data.set_index(keys=['oppfieldhist_CreatedDate'], inplace=True)
            data.sort_index(axis=1, ascending=True, inplace=True)
            sys.stdout.write('\n\n -*- {0}Opportunity Field History Records{1} -*- \n\n'.format('\x1b[6;30;46m','\x1b[0m'))
            print(data.info())
            return data
        
        except Exception as error:   
            sys.stdout.write('\n {0} *** Error: <sf_Manager.get_opportunity_field_history()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc())) 
            return pd.DataFrame()

    def get_opportunity_history(self):
        '''
        Queries SalesForce Opportunity History Object and returns results as Dask DataFrame object
        
        Parameters:
            None
       
        Returns:
            Data (Dask DataFrame): Opportunity History Records (Large DataSet)
        ''' 
        try:
            data = pd.DataFrame()
            sys.stdout.write('\n\n******** {0} Getting Opportunity History Data {1}  ******** \n'.format('\x1b[6;30;43m', '\x1b[0m'))
            
            # For later, update the SOQL to return only records which are updated since last call; i.e., add WHERE condition to the below SOQL for updatedDate of records. Ortherwise, it will pull all 1.1M rows each time
            soql_query = "SELECT " + self.OpportunityHistory_fields + " FROM OpportunityHistory"
            data = sf.dict_to_df(soql_result=sf.query(query=soql_query))
            data.columns = 'opphist_' + data.columns
            
            # Convert to DateTime: VERY IMPORTANT for fast processing
            date_labels = ['opphist_CreatedDate']
            data[date_labels] = data[date_labels].apply(pd.to_datetime)
            data['opphist_CreatedDate_'] = data['opphist_CreatedDate']
            data.sort_values(by=['opphist_CreatedDate'], ascending=[True], inplace=True)
            data.set_index(keys=['opphist_CreatedDate'], inplace=True)
            
            data.sort_index(axis=1, ascending=True, inplace=True)
            sys.stdout.write('\n\n -*- {0}Opportunity History Records{1} -*- \n\n'.format('\x1b[6;30;46m','\x1b[0m'))
            print(data.info())
            
            # Export Data to Google Drive, as parquet GZip file
            fpath = GOOGLE_PATH + 'Shareddrives/UB/Sales/Sales & Solution Engineers/Sales/Analysis/data/opps_cases/opp_history.parquet.gzip'
            data.to_parquet(path=fpath, index=True)
            sys.stdout.write('\n\n -*- {0}Exporting Parquet File: {2} . . . {1} -*- \n\n'.format('\x1b[6;30;46m','\x1b[0m', fpath))
            
            # Returns Dask Dataframe
            return dd.from_pandas(data, npartitions=5)
        
        except Exception as error:   
            sys.stdout.write('\n {0} *** Error: <sf_Manager.get_opportunity_history()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc())) 
            return pd.DataFrame() 
        
    def get_opportunity_products(self):
        '''
        Returns SalesForce Opportunity Line Items (i.e., Products)
        
        Parameters:
            None
       
        Returns:
            Data (Pandas DataFrame): Opportunity Line Items (i.e., Products)
        ''' 
        try:
            data = pd.DataFrame()
            sys.stdout.write('\n\n******** {0} Getting Opportunity Line Items (i.e., Products) Data {1}  ******** \n'.format('\x1b[6;30;43m', '\x1b[0m'))
            soql_query = soql = "SELECT Id, CreatedDate, OpportunityId, Product2Id, ProductCode, Quantity, TotalPrice, Annual_per_User_License_Cost__c, Live_Annual_Recurring_Revenue__c, Term_Start_Date__c, Term_End_Date__c FROM OpportunityLineItem"
            data = sf.dict_to_df(soql_result=sf.query(query=soql_query))
            data.columns = 'oppli_' + data.columns
        
            sys.stdout.write('\n\n -*- {0}Opportunity Line Items (i.e., Products) Records{1} -*- \n\n'.format('\x1b[6;30;46m','\x1b[0m'))       
            print(data.info())
 
            return data
        
        except Exception as error:   
            sys.stdout.write('\n {0} *** Error: <sf_Manager.get_opportunity_products()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc())) 
            return pd.DataFrame()  
        
    def get_product_details(self):
        '''
        Returns SalesForce Product2 Records (i.e., ['Subscription Services', None, 'UPro', 'Language Pack', 'Services', 'Core', 'EY Skills Foundry', 'Non-Subscription Services', 'Content'])
        
        Parameters:
            None
       
        Returns:
            Data (Pandas DataFrame): SalesForce Product2 Records (i.e., UB Product types such as UPro)
        ''' 
        try:
            data = pd.DataFrame()
            sys.stdout.write('\n\n******** {0} Getting Products Data {1}  ******** \n'.format('\x1b[6;30;43m', '\x1b[0m'))
            soql_query = soql = "SELECT Id, Name, Family FROM Product2"
            data = sf.dict_to_df(soql_result=sf.query(query=soql_query))
            data.columns = 'prod_' + data.columns
        
            sys.stdout.write('\n\n -*- {0}Opportunity Products Records{1} -*- \n\n'.format('\x1b[6;30;46m','\x1b[0m'))       
            print(data.info())
 
            return data
        
        except Exception as error:   
            sys.stdout.write('\n {0} *** Error: <sf_Manager.get_opportunity_products()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc())) 
            return pd.DataFrame()
        
    def get_cases(self, soql_query="SELECT Id FROM Case LIMIT 1"):
        
        '''
        Queries SalesForce API for Case data
        
        Parameters:
            soql_query (string): SalesForce Object Query Language query
        
        Returns: 
            data (Pandas DataFrame): SalesForce Case Records 
        '''        
        
        try:
            data = pd.DataFrame()
            sys.stdout.write('\n******** {0}Getting Case Data{1}  ******** \n'.format('\x1b[6;30;43m', '\x1b[0m'))
            data = sf.dict_to_df(soql_result=sf.query(query=soql_query))
            data.columns = 'case_' + data.columns
            data.sort_index(axis=1, ascending=True, inplace=True)
            
            # Print results
            sys.stdout.write('\n\n -*- {0}Case Records{1} -*- \n\n'.format('\x1b[6;30;46m','\x1b[0m'))
            print(data.info())
            return data
        
        except Exception as error:
            sys.stdout.write('\n {0} *** Error in <sf_Manager.get_cases()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))
            return pd.DataFrame()
        
    def get_ironclad_workflows(self):
        
        '''
        Queries SalesForce IronClad Workflows and related Cases
        
        Parameters:
            None 
        
        Results: 
            data (Pandas DataFrame): SalesForce IronClad Workflow Cases
        '''        
        try:
    
            # Get IronClad Workflows
            sys.stdout.write('\n ********  {0} Getting iron_clad_workflows Data {1}  ******** \n'.format('\x1b[6;30;43m', '\x1b[0m'))
            soql_query = "SELECT iw.id, iw.Name, iw.CreatedDate, iw.ironclad__Workflow_ID__c, iw.ironclad__Workflow_Name__c, iw.ironclad__Workflow_Step__c, iw.ironclad__Workflow_Link__c, iw.Case__c FROM ironclad__Ironclad_Workflow__c iw WHERE iw.Case__c != null"
            IW = self.dict_to_df(soql_result=self.query(query=soql_query))
            IW.columns = 'ironclad_workflow_' + IW.columns
            print(IW.info())
            
            # Get IronClad Workflow Approvals
            sys.stdout.write('\n ********  {0} Getting ironclad__Ironclad_Approval__c {1}  ******** \n'.format('\x1b[6;30;43m', '\x1b[0m'))
            soql = "SELECT ia.Id, ia.IsDeleted, ia.Name, ia.CurrencyIsoCode, ia.CreatedDate, ia.CreatedById, ia.LastModifiedDate, ia.LastModifiedById, ia.SystemModstamp, ia.LastActivityDate, ia.ironclad__Ironclad_Workflow__c, ia.ironclad__Approval_Name__c, ia.ironclad__Approval_Role__c, ia.ironclad__Approval_Status__c FROM ironclad__Ironclad_Approval__c ia"
            IA = sf.dict_to_df(soql_result=sf.query(query=soql))
            IA.columns = 'ironclad_workflow_approval_' + IA.columns
            IA.sort_index(axis=1, ascending=True, inplace=True)
            print(IA.info())
            
            # Merge Approvals with Workflows
            sys.stdout.write('\n ********  {0} Merging iron_clad_workflows & ironclad__Ironclad_Approval__c {1}  ******** \n'.format('\x1b[6;30;43m', '\x1b[0m'))
            aggs_ = pd.merge(IW, IA, how='left', left_on='ironclad_workflow_Id', right_on='ironclad_workflow_approval_ironclad__Ironclad_Workflow__c')
            aggs_.drop_duplicates(subset=['ironclad_workflow_Id', 'ironclad_workflow_approval_Id'], inplace=True)
            aggs_.sort_index(axis=1, ascending=True, inplace=True)
            print(aggs_.info())
            del IW, IA
            return aggs_
        
        except Exception as error:
            sys.stdout.write('\n {0} *** Error in <sf_Manager.get_ironclad_workflows()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))
            return pd.DataFrame()
        
    def get_clean_merged_data(self, accounts=pd.DataFrame(), opportunities=pd.DataFrame(), cases=pd.DataFrame()):
        
        '''
        Cleans and Merges SalesForce Accounts, Opportunities, and Cases records. 

            Parameters:
                accounts (Pandas DataFrame object): SalesForce Account Records retrieved from SFDC API (SOQL) as a Pandas DataFrame
                opportunities (Pandas DataFrame object): SalesForce Opportunity Records retrieved from SFDC API (SOQL) as a Pandas DataFrame
                cases (Pandas DataFrame object): SalesForce Case Records retrieved from SFDC API (SOQL) as a Pandas DataFrame
             
            Returns:
               1. account_opps_cases (Pandas DataFrame): aggs_ Account, Opportunity, Case records, and matching Legal IronClad Workflows, and type of Udemy Product tied to each Opportunity 
        '''       

        try:
            # Main DataFrame for all aggs_ data
            account_opps = pd.DataFrame()
            
            # Additional SalesForce Data Merge with Account, Opportunity, and Case records
            users = self.get_users()
            iron_clad_workflows = self.get_ironclad_workflows()
            opportunity_products = self.get_opportunity_products()
            product_details = self.get_product_details()

            # -*- -------------------ACCOUNT DATA ------------------- -*-

            sys.stdout.write('\n\n{0} Cleaning Accounts Data {1}\n'.format('\x1b[6;30;43m', '\x1b[0m'))
            # To retrieve Column index by location; not used elsewhere in this Class at the moment;
            idx_dic = {}
            for col in accounts.columns:
                idx_dic[col] = accounts.columns.get_loc(col)
                
            # Clean fields with NaN
            acct_values = {
                            'acct_NumberOfEmployees': 0, 'acct_AnnualRevenue': 0,'acct_Employee_Segment__c': 'nan',
                            'acct_Region_New__c': 'nan', 'acct_Region__c': 'nan', 'acct_Business_Segment__c': 'nan'
                          }
            accounts.fillna(value=acct_values, inplace=True)
            
            # Convert to int32
            accounts = accounts.astype({'acct_NumberOfEmployees': 'int32'}).copy()
            
            # Clean dirty data df.loc[df.column_a == 44, 'column_b'] = 100
            is_acct_Region_New__c_India = (accounts.acct_Region_New__c == 'India')
            accounts.loc[idx[is_acct_Region_New__c_India], 'acct_Region_New__c'] = 'APAC'
            is_acct_Region_New__c_US_West = (accounts.acct_Region_New__c == 'US–West')
            accounts.loc[idx[is_acct_Region_New__c_US_West], 'acct_Region_New__c'] = 'AMER'
            is_acct_Business_Segment__c_Commericial = (accounts.acct_Business_Segment__c == 'Commericial')
            accounts.loc[idx[is_acct_Business_Segment__c_Commericial], 'acct_Business_Segment__c'] = 'Commercial'
           
            # Convert to Categorical Data Type: faster performance
            categorical_labels = ['acct_Region_New__c', 'acct_Employee_Segment__c', 'acct_Business_Segment__c']
            accounts[categorical_labels] = accounts[categorical_labels].apply(pd.Categorical)
            
            # -*- -------------------- OPPORTUNITY DATA ------------------- -*-

            # Clean fields with NaN
            sys.stdout.write('\n\n{0} Cleaning Opportunities Data {1}\n'.format('\x1b[6;30;43m', '\x1b[0m'))
            missing_values = {
                            'opp_SBCF_Prior_License_Qty__c': 0, 'opp_Type': 'none', 'opp_FiscalYear': 0,
                            'opp_FiscalQuarter': 0, 'opp_CloseDate': '1900-01-01', 'opp_StageName': 'none', 
                            'opp_Contract_Length__c': 1, 'opp_Contract_Length_CPQ__c': 1, 'opp_Amount': 0, 
                            'opp_SBCF_Current_License_Qty__c': 0, 'opp_CreatedDate': '1900-01-01', 'opp_Opportunity_Name_Append__c': 'none', 
                            'opp_NewARR__c': 0, 'opp_RenewalARR__c': 0, 'opp_New_Bookings_Local_Currency__c': 0
                         }
            opportunities.fillna(value=missing_values, inplace=True)

            # Convert to DateTime
            opp_date_labels = ['opp_CloseDate', 'opp_CreatedDate', 'opp_License_Start_Date_New__c',  'opp_License_End_Date_New__c']
            opportunities[opp_date_labels] = opportunities[opp_date_labels].apply(pd.to_datetime)
            # maybe use this to convert to datetime instead: np.array(['2001-01-01T12:00', '2002-02-03T13:56:03.172'], dtype='datetime64')
            
            # Convert Opp ARR in INR currency to USD
            opp_int_labels = ['opp_FiscalYear', 'opp_FiscalQuarter', 'opp_SBCF_Prior_License_Qty__c', 'opp_Contract_Length__c', 'opp_Contract_Length_CPQ__c']
            opportunities[opp_int_labels] = opportunities[opp_int_labels].apply(pd.to_numeric)
            
            # Sort Opp Stages and Convert to Categorical: faster processing and indexing
            categorical_labels = ['opp_StageName', 'opp_Type']
            opportunities.opp_StageName = pd.Categorical(opportunities.opp_StageName, categories=sorted(list(opportunities.opp_StageName.unique())), ordered=True)
            opportunities[categorical_labels] = opportunities[categorical_labels].apply(pd.Categorical)
            
            # Add new ARR fields for USD Converted ARR
            opp_arr_labels = ['opp_NewARR__c', 'opp_RenewalARR__c', 'opp_New_Bookings_Local_Currency__c']
            is_opp_CurrencyIsoCode_INR =  opportunities.loc[opportunities.opp_CurrencyIsoCode == 'INR'].index
            for opp_arr_label in opp_arr_labels: 
                new_arr_label = '{}_Converted__USD'.format(str(opp_arr_label))
                opportunities.loc[:, new_arr_label] = opportunities.loc[:, opp_arr_label]
                opportunities.loc[idx[is_opp_CurrencyIsoCode_INR], new_arr_label] = (opportunities.loc[idx[is_opp_CurrencyIsoCode_INR], new_arr_label] * self.inr_to_usd)
            
                        
            # New fields for Opp/Case Created/Closed Date [Year, Quarter, Week]
            opportunities['opp_CreatedDate_Year'] = opportunities.opp_CreatedDate.dt.year
            opportunities['opp_CreatedDate_Quarter'] = opportunities.opp_CreatedDate.dt.quarter
            opportunities['opp_CreatedDate_Week'] = opportunities.opp_CreatedDate.dt.isocalendar().week
            opportunities['opp_CloseDate_Year'] = opportunities.opp_CloseDate.dt.year
            opportunities['opp_CloseDate_Quarter'] = opportunities.opp_CloseDate.dt.quarter
            opportunities['opp_CloseDate_Week'] = opportunities.opp_CloseDate.dt.isocalendar().week
            
            # Convert to Int
            opp_int_labels = ['opp_CreatedDate_Year', 'opp_CreatedDate_Quarter', 'opp_CreatedDate_Week', 
                              'opp_CloseDate_Year', 'opp_CloseDate_Quarter', 'opp_CloseDate_Week']
            opportunities[opp_int_labels] = opportunities[opp_int_labels].apply(pd.to_numeric) 
            
            # -*- ------------------- CASE DATA ------------------- -*-
            
            # Clean fields with NaN
            sys.stdout.write('\n\n{0} Cleaning Cases Data {1}\n'.format('\x1b[6;30;43m', '\x1b[0m'))
            case_values = {
                            'case_CreatedDate': '1900-01-01',
                            'case_Creator_Role__c': 'None'
                          }
            cases.fillna(value=case_values, inplace=True)
            
            # Convert to DateTime
            case_date_labels = ['case_CreatedDate']
            cases[case_date_labels] = cases[case_date_labels].apply(pd.to_datetime)         
         
            # Convert Opp ARR in INR currency to USD
            cases['case_CreatedYear'] = cases.case_CreatedDate.dt.year
            cases['case_CreatedQuarter'] = cases.case_CreatedDate.dt.quarter
            cases['case_CreatedWeek'] = cases.case_CreatedDate.dt.isocalendar().week
            
            # Convert to Int
            case_int_labels = ['case_CaseNumber', 'case_CreatedYear', 'case_CreatedQuarter', 'case_CreatedWeek']
            cases[case_int_labels] = cases[case_int_labels].apply(pd.to_numeric)  

            # Create S&SE Case Creator Team Values
            is_AE = cases.case_Creator_Role__c.str.contains('AE|SE|SAE|ADR|SDR|UFB Mgmt|Rev Ops|Deactivated')
            is_CS = cases.case_Creator_Role__c.str.contains('CS|CSM|CSA|ADR|SDR')
            is_NBV = cases.case_Creator_Role__c.str.contains('Udemy NV|UFG')
            is_UG = cases.case_Creator_Role__c.str.contains('UFG')
            is_BD = cases.case_Creator_Role__c.str.contains('Partnerships')
            is_support = cases.case_Creator_Role__c.str.contains('Sales Customer Experience')

            cases['case_Creator_Team'] = 'Sales'
            cases.loc[idx[is_CS], 'case_Creator_Team'] = 'Customer Success'
            cases.loc[idx[is_NBV], 'case_Creator_Team'] = 'NBV'
            cases.loc[idx[is_UG], 'case_Creator_Team'] = 'UG'
            cases.loc[idx[is_BD], 'case_Creator_Team'] = 'BD'
            cases.loc[idx[is_support], 'case_Creator_Team'] = 'UB Support'
            
            # -*- ------------------- MERGE DATA ------------------- -*-
            
            '''
            Create New aggs_ DataFrames:
                1. account_opps = Accounts + Opportunities
                2. account_opps_cases = account_opps + cases
                3. case_opps = cases + account_opps
            '''
            
            # 1. Merge Opps with Accounts (account_opps); remove dups, and drop the opp_Account data field (not needed after )
            sys.stdout.write('\n\n{0} Merging Accounts, Opportunities, and Cases Records (acct_opps) {1}\n'.format('\x1b[6;30;43m', '\x1b[0m'))
            account_opps = pd.merge(opportunities, accounts, how='left', left_on='opp_AccountId', right_on='acct_Id')
            account_opps.drop_duplicates(subset=['opp_Id'], inplace=True)
            account_opps.drop(columns=['opp_Account'], axis=1, inplace=True) 
            
            # 2. Merge account_opps with Cases; account_opps_cases
            account_opps_cases = pd.merge(account_opps, 
                                               cases[['case_CreatedYear','case_CreatedQuarter', 'case_CreatedWeek', 'case_Id', 
                                                      'case_Related_Opportunity__c','case_CaseNumber', 'case_OwnerId', 'case_Status', 
                                                      'case_AE__c', 'case_Type', 'case_Sub_Type__c', 'case_Description', 'case_Subject',
                                                      'case_Creator_Team', 'case_CreatedDate', 'case_Oppty_Stage__c', 'case_CreatedById', 'case_RecordTypeName__c']], 
                                               how='left', 
                                               left_on='opp_Id', 
                                               right_on='case_Related_Opportunity__c'
                                              )
            # 3. Merge User Name for case_OwnerId with account_opps_cases; drop dups, rename columns, and drop duplicate columns
            account_opps_cases = pd.merge(account_opps_cases, users[['user_Id', 'user_Name']], how='left', left_on='case_OwnerId',  right_on='user_Id')
            account_opps_cases.drop_duplicates(subset=['opp_Id', 'case_Id'], inplace=True)
            account_opps_cases.rename(columns={'user_Name':'case_SSEOwnerName'}, inplace=True)
            account_opps_cases.drop(columns=['user_Id'], axis=1, inplace=True)        

            # 3. Merge User Name for case_CreatedById with account_opps_cases; drop dups, rename columns, and drop duplicate columns
            account_opps_cases = pd.merge(account_opps_cases, users[['user_Id', 'user_Name']], how='left', left_on='case_CreatedById',  right_on='user_Id')
            account_opps_cases.drop_duplicates(subset=['opp_Id', 'case_Id'], inplace=True)
            account_opps_cases.rename(columns={'user_Name':'case_CreatedByName'}, inplace=True)
            account_opps_cases.drop(columns=['user_Id'], axis=1, inplace=True)   

            # 5. Merge User Name for opp_OwnerId, drop dups, and then rename columns
            account_opps_cases = pd.merge(account_opps_cases, users[['user_Id', 'user_Name', 'user_Manager_for_Reports__c', 'user_Email']], how='left', left_on='opp_OwnerId',  right_on='user_Id')
            account_opps_cases.drop_duplicates(subset=['opp_Id', 'case_Id'], inplace=True)
            account_opps_cases.rename(columns={'user_Name': 'opp_OwnerName', 'user_Manager_for_Reports__c': 'opp_AE_Manager', 'user_Email': 'opp_OwnerEmail'}, inplace=True)
            account_opps_cases.drop(columns=['user_Id'], axis=1, inplace=True) 
          
            # 6. Merge IronClad data
            account_opps_cases = pd.merge(account_opps_cases, iron_clad_workflows, how='left', left_on='case_Id',  right_on='ironclad_workflow_Case__c')
            account_opps_cases.drop_duplicates(subset=['opp_Id', 'case_Id'], inplace=True)           
            
            # 7. Merge Opp Line Items (i.e., Product) and Product Details
            sys.stdout.write('\n\n{0} Merging Opportunity Line Items (i.e., Products) with Line Item Product Details {1}\n'.format('\x1b[6;30;43m', '\x1b[0m'))
            temp_merge = pd.merge(opportunity_products, product_details, how='left', left_on='oppli_Product2Id', right_on='prod_Id')
            temp_merge.drop_duplicates(subset=['oppli_Id'], inplace=True)
            account_opps_cases = pd.merge(account_opps_cases, temp_merge, how='left', left_on='opp_Id',  right_on='oppli_OpportunityId')
            account_opps_cases.drop_duplicates(subset=['opp_Id', 'case_Id'], inplace=True)
            

            # Get S&SE Assignment Rules and add Assigned S&SE attribute/column ('opp_SE_Assigned')
            sse_assignments = self.sse_assignments
            account_opps_cases['opp_SE_Assigned']='None'
            for region in sse_assignments.keys():
                if "AMER" in region: 
                    for sse in sse_assignments[region][0].keys():
                        if 'Seth Klein' not in sse:
                            criteria = account_opps_cases.opp_Live_UFB_Owner_String__c.str.contains("|".join(sse_assignments[region][0][sse][0]['opp_Live_UFB_Owner_String__c']))
                            idx_keys = account_opps_cases.loc[criteria].index
                            account_opps_cases.loc[idx[idx_keys], 'opp_SE_Assigned'] = sse 
                            
                if "EMEA" in region: 
                    for sse in sse_assignments[region][0].keys():
                        if 'Garry Hickey' not in sse:
                            criteria = ((account_opps_cases.acct_Sub_Region__c.str.contains("|".join(sse_assignments[region][0][sse][0]['acct_Sub_Region__c']))) 
                                       & account_opps_cases.opp_Live_UFB_Owner_String__c.str.contains("|".join(sse_assignments[region][0][sse][0]['opp_Live_UFB_Owner_String__c'])))
                            idx_keys = account_opps_cases.loc[criteria].index
                            account_opps_cases.loc[idx[idx_keys], 'opp_SE_Assigned'] = sse
                if "APAC" in region: 
                    for sse in sse_assignments[region][0].keys():
                        if 'Steven Reynolds' not in sse:
                            criteria = ((account_opps_cases.acct_BillingCountry.str.contains("|".join(sse_assignments[region][0][sse][0]['acct_BillingCountry']))) 
                                       & account_opps_cases.opp_Live_UFB_Owner_String__c.str.contains("|".join(sse_assignments[region][0][sse][0]['opp_Live_UFB_Owner_String__c'])))
                            idx_keys = account_opps_cases.loc[criteria].index
                            account_opps_cases.loc[idx[idx_keys], 'opp_SE_Assigned'] = sse 
           
            # Add Opportunity Contract Length
            account_opps_cases['opp_Contract_Length_CPQ__c_Years'] = 0
            account_opps_cases.loc[((account_opps_cases.opp_Contract_Length_CPQ__c >0) & (account_opps_cases.opp_Contract_Length_CPQ__c <=18)), ['opp_Contract_Length_CPQ__c_Years']] = 1
            account_opps_cases.loc[((account_opps_cases.opp_Contract_Length_CPQ__c >18) & (account_opps_cases.opp_Contract_Length_CPQ__c <=30)), ['opp_Contract_Length_CPQ__c_Years']] = 2
            account_opps_cases.loc[((account_opps_cases.opp_Contract_Length_CPQ__c >30) & (account_opps_cases.opp_Contract_Length_CPQ__c <=100)), ['opp_Contract_Length_CPQ__c_Years']] = 3
            
            # Clean Missing data
            missing_values = {'opp_FiscalYear': 0, 'opp_FiscalQuarter': 0, 'case_CreatedYear': 0, 'case_CreatedQuarter': 0,
                              'opp_SBCF_Prior_License_Qty__c': 0, 'case_CaseNumber': 0, 'case_CreatedWeek': 0, 
                              'case_SSEOwnerName': 'none', 'case_Status': 'none', 
                              'opp_ForecastCategory': 'none', 'prod_Name': 'none', 'opp_UFB_Sales_Owner_Segment__c': 'none',
                              'prod_Family': 'none', 'oppli_ProductCode': 'none', 'acct_Customer_Tier__c': 'none', 
                              'opp_UFB_Sales_Owner_Team__c': 'none', 'opp_UFB_Sales_Owner_Region__c': 'none', 
                              'opp_UFB_Sales_Owner_Title__c': 'none', 'acct_CSM_Role__c': 'none', 'acct_Current_LMS__c': 'none',
                              'case_Subject': 'none', 'acct_Region_Detail__c': 'none', 'acct_fcio1__Territory__c': 'none', 'acct_ZoomInfo_Industry__c': 'none',
                              'case_Creator_Team': 'none', 'case_Oppty_Stage__c': 'none', 'acct_Partner_Type__c': 'none', 'case_Sub_Type__c': 'none'
                             }
            account_opps_cases.fillna(value=missing_values, inplace=True)       
            
            # Convert int64 to int32 Columns
            cast_cols = dict()
            cols = list()
            cols = list(account_opps_cases.dtypes[account_opps_cases.dtypes == 'int64'].index)
            for col in cols:
                cast_cols[col] = 'int32'
            account_opps_cases = account_opps_cases.astype(cast_cols)
            
            # Convert float64 to float32 Columns
            cast_cols = dict()
            cols = list()
            cols = list(account_opps_cases.dtypes[account_opps_cases.dtypes == 'float64'].index)
            for col in cols:
                cast_cols[col] = 'float32'
            account_opps_cases = account_opps_cases.astype(cast_cols) 
            
            # Convert to Categorical Values: decreases DataFrame size!
            categorical_labels = ['acct_Region__c', 'acct_Business_Segment__c', 'opp_Opportunity_Owner_Role__c', 'opp_Live_UFB_Owner_String__c', 
                                  'opp_OwnerEmail', 'opp_OwnerId', 'opp_RecordTypeId', 'opp_OwnerName', 'opp_Opportunity_Owner_ID__c', 'acct_Industry', 
                                  'acct_Account_Owner_Text__c', 'opp_Product_Type__c', 'opp_AE_Manager', 'acct_ProductType_f__c', 'opp_SE_Assigned', 
                                  'acct_Sub_Region__c', 'acct_BillingCountry', 'acct_Type', 'opp_ForecastCategoryName', 'opp_ForecastCategory', 'opp_CS_Segment__c', 
                                  'case_SSEOwnerName', 'acct_Region_text__c', 'case_Status', 'acct_Account_Record_Type_Name__c', 'prod_Name', 'opp_CurrencyIsoCode', 
                                  'acct_CurrencyIsoCode', 'opp_UFB_Sales_Owner_Segment__c', 'prod_Family', 'oppli_ProductCode', 'acct_Customer_Tier__c', 
                                  'opp_UFB_Sales_Owner_Team__c', 'opp_UFB_Sales_Owner_Region__c', 'opp_UFB_Sales_Owner_Title__c', 'acct_CSM_Role__c', 
                                  'acct_Current_LMS__c', 'case_Subject', 'acct_Region_Detail__c', 'acct_fcio1__Territory__c', 'acct_ZoomInfo_Industry__c', 
                                  'case_Creator_Team', 'case_Type', 'case_Sub_Type__c', 'case_Oppty_Stage__c', 'acct_Partner_Type__c']
            
            account_opps_cases[categorical_labels] = account_opps_cases[categorical_labels].apply(pd.Categorical)
            
            # Sort opp_CloseDate, Set Index (fast slicing), and Sort Columns labels (axis=1)
            index = ['opp_FiscalYear', 'opp_FiscalQuarter', 'opp_StageName', 'opp_Type', 
                     'acct_Region_New__c', 'acct_Region__c', 'acct_Business_Segment__c',
                     'case_CreatedYear','case_CreatedQuarter']
            
            # Cast DataFrame objects for better performance before indexing 
            cast_cols = {'opp_FiscalYear': 'int16', 'opp_FiscalYear': 'int16', 'case_CreatedYear': 'int16', 'case_CreatedQuarter': 'int16'}
            account_opps_cases = account_opps_cases.astype(cast_cols)
            
            account_opps_cases.sort_values(by=['opp_CloseDate'], ascending=True, inplace=True)
            account_opps_cases.set_index(index, inplace=True)
            account_opps_cases.sort_index(axis=1, ascending=True, inplace=True)
              
            self.acct_opps_arr_columns = [col for col in account_opps_cases.columns if 'ARR' in col]
            self.export_to_parquet(data=account_opps_cases, fname='acct_opps.parquet.gzip')
            
            sys.stdout.write('\n{0} -------- DONE --------  {1}\n'.format('\x1b[32m', '\x1b[32m'))
            del accounts, opportunities, cases, iron_clad_workflows, users, opportunity_products, product_details, temp_merge
            return account_opps_cases

        except Exception as error:
            sys.stdout.write('\n {0} *** Error: <sf_Manager.get_clean_aggs__data()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))
            return pd.DataFrame()

    def get_merged_opp_case_history_stages(self, left_data=pd.DataFrame(), right_data=pd.DataFrame(), right_by=''):

        '''
        Match Opportunity Stage cloases to the created Date of each Opportunity Case

            Parameters:
               1. left_data (Pandas DataFrame): DataFrame to Match with right; must contained a matching sorted index with right
               2. right_data (Pandas DataFrame): DataFrame to Match with left; must contained a matching sorted index with left
               3. right_by (string): Column to use as lookup from right DataFrame with Left DataFrame
            
            Returns:
               1. matches (Pandas DataFrame): aggs_ with matches on OppStage Name at time Case was Created
               Index(['oppfieldhist_CreatedById', 'oppfieldhist_CreatedDate', 'oppfieldhist_DataType', 'oppfieldhist_Field', 'oppfieldhist_Id', 'oppfieldhist_IsDeleted', 'opphist_StageName', 'oppfieldhist_OldValue', 'right_by']
        '''       

        try:
            
            # right = self.get_opportunity_history()

            # Copy Opps with S&SE Cases to a new DataFrame; subset of required fields only
            left = (left_data.loc[(left_data.case_Id.notnull()), 
                                  ['opp_Id', 'case_Id', 'case_CreatedDate']].reset_index(drop=True).copy())
            
            # Set Time Series index.  MUST be DateTime UTC, and MUST be Sorted; 
            left['case_CreatedDate'] = left['case_CreatedDate'].apply(pd.to_datetime)
            left.sort_values(by=['case_CreatedDate'], ascending=[True], inplace=True)
            left.set_index('case_CreatedDate', inplace=True)
            
            # Filter Opp History Records to match records of Opps with S&SE Cases
            right = (right_data.loc[right_data[right_by].isin(left.opp_Id.unique()), :].copy())

            # For roughly 10K records in both left and right DataFrames, it will take 5 minutes to match records based on closest TimeSeries index.
            (matched_results, unmatched_results) = self.merge_asof(left=left, right=right, left_by=['opp_Id'], right_by=right_by)

            # Merge matches
            # [['opp_Id', 'case_Id'] + right_columns]
            index_ = left_data.index
            matches = (left_data.merge(matched_results, how='left', left_on=['opp_Id', 'case_Id'],  right_on=['opp_Id', 'case_Id'], suffixes=('_left', '_right')))
            matches.set_index(index_, inplace=True)

            # -*- Calculate S&SE New and Renewall ARR Impact & Merge with matches -*- 
            df = pd.DataFrame()
            pvt = pd.DataFrame()
            df = matches.loc[(matches.case_Id.notnull())].copy()
            
            # Better performance for getting/setting via sorted index
            df.reset_index(inplace=True)
            df.sort_index(inplace=True)

            # -*- Get Aggregate Number of S&SE Cases, grouped by 'Opp_Id' 'case_OwnerId' (i.e., the S&SE Case Owner's SalesForce ID) -*-
            values = ['case_Id']
            pvt = pd.DataFrame()
            pvt = (pd.pivot_table(data=df.reset_index(), 
                                   values=values, 
                                   index=['opp_Id', 'case_OwnerId'], 
                                   columns=None,
                                   aggfunc={values[0]: np.count_nonzero}, 
                                   dropna=True,
                                   fill_value=0
                                  )
                   )

            opp_grp_total = pvt.groupby(level=[0]).transform('sum') # Group by Opp_Id
            opp_sse_grp_total = pvt.groupby(level=[0, 1]).transform('sum') # Group by Opp_Id and case_OwnerId

            # Merge Total No. of S&SE Cases for each Opp and each Opp and S&SE
            aggs_ = pd.DataFrame()
            aggs_ = (pd.merge(left=opp_grp_total, right=opp_sse_grp_total, how='left', left_index=True , right_index=True, suffixes=('_left', '_right')))
            aggs_.rename(columns={'case_Id_left': 'case_SSE_Total_Cases', 'case_Id_right': 'case_SSE_Total_Opp_Cases'}, inplace=True)
            aggs_['case_SSE_Percentage_Impact'] = aggs_.case_SSE_Total_Opp_Cases/aggs_.case_SSE_Total_Cases

            # Merge with matches and calculate S&SE Revenue Impact for Opps engaged
            matches = pd.merge(matches, aggs_, how='left', left_on=['opp_Id', 'case_OwnerId'], right_index=True,  suffixes=('_left', '_right'))
            matches['case_SSE_NARR_Contribution'] = matches.opp_NewARR__c_Converted__USD * matches.case_SSE_Percentage_Impact
            matches['case_SSE_RARR_Contribution'] = matches.opp_RenewalARR__c_Converted__USD * matches.case_SSE_Percentage_Impact
            matches['opp_Weeks_to_Close'] = (matches.opp_CloseDate.dt.isocalendar().week.values) - (matches.opp_CreatedDate.dt.isocalendar().week.values)

            # Align the Legacy Opp Stage Name Values with the New Stage Name Values
            matches.loc[matches.opphist_StageName=='AE Qualified', 'opphist_StageName'] = '2-Qualified'
            matches.loc[matches.opphist_StageName.isin(['Business Requirements Identified', 'Business requirements identifed']), 'opphist_StageName'] = '3-Business Requirements'
            matches.loc[matches.opphist_StageName=='Finalizing Closure', 'opphist_StageName'] = '7-Finalizing Closure'
            matches.loc[matches.opphist_StageName=='Finance Review', 'opphist_StageName'] = '8-Finance Review'
            matches.loc[matches.opphist_StageName.isin(['Kick back to SDR/BDR', 'Lead from SDR/BDR']), 'opphist_StageName'] = '1-Pre-Qualified'
            matches.loc[matches.opphist_StageName=='Negotiation/Procurement/Legal', 'opphist_StageName'] = '6-Negotiation'
            matches.loc[matches.opphist_StageName=='Proof of Concept (POC)', 'opphist_StageName'] = '4-Solution Design'
            matches.loc[matches.opphist_StageName=='Proposal/Price Presented', 'opphist_StageName'] = '5-Proposal'
            matches.sort_index(axis=1, ascending=True, inplace=True)
            
            del left_data, right, right_data, df, pvt, opp_grp_total, opp_sse_grp_total, aggs_, matched_results, unmatched_results, index_
            return matches
        
        except Exception as error:
            sys.stdout.write('\n {0} *** Error: <sf_Manager.get_aggs__opp_case_history_stages()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))
            return pd.DataFrame()

    def merge_asof(self, left=pd.DataFrame(), right=pd.DataFrame(), left_by=['case_CreatedDate'], right_by=['opphist_CreatedDate']):
        '''
        Match closest Opp History record by created Date with SalesForce Case by Created date, and by Opp Id. For large datasets (>100K records), consider refactoring this method with Dask DataFrames for parallelization

            Parameters:
                left (Pandas DataFrame object): SalesForce Records to match with right dataframe; must contain sorted TimeSeries field
                right (Pandas DataFrame object): SalesForce Records to match with left; must contain roted TimeSeries index
            
            Returns:
               1. matched_results (Pandas DataFrame): matched records from left DataFrame
               2. unmatched_results (Pandas DataFrame): unmatched records from left DataFrame
        '''       

        try:
            
            # Initialize DataFrames for matches
            matches = pd.DataFrame()
            matched_results = pd.DataFrame()
            unmatched_results = pd.DataFrame()
            
            # Results of the merge_asof matches
            self.results = {'merge_asof_results_backward': set(), 'merge_asof_results_forward': set()}
            
            # Print Number of Records from Left (S&SE Cases) and Right (Opp History) DataFrames
            sys.stdout.write('\n{0} Records to Match {1} \n\n'.format('\x1b[6;30;43m', '\x1b[0m'))
            sys.stdout.write('\n{0}\n'.format(left.info()))
            sys.stdout.write('\n\n{0} Against Opp History Records {1} \n\n'.format('\x1b[6;30;43m', '\x1b[0m'))
            sys.stdout.write('\n{0}\n'.format(right.info()))
            
            # Set initial merge_asof dearch direction
            max_days = (left.index.max() - left.index.min()).days
            max_tolerance = list(range(1, max_days))
            tolerance = 1
            search_direction = 'nearest'
            sys.stdout.write('\n{0} Cases Span Number of Days: {1} {2} \n\n'.format('\x1b[6;30;43m', max_days, '\x1b[0m'))
            
            # Iterate through number of days in the left (S&SE Cases) DataFrame
            for tolerance in max_tolerance:
                # Set search direction to 'backward' following initial 1 day 'nearest' matches; 
                if tolerance > 1:
                    search_direction = 'backward'
                matches = (pd.merge_asof(left, right, 
                                         left_index=True, right_index=True, 
                                         left_by=left_by, right_by=right_by, 
                                         tolerance=pd.Timedelta('{0}d'.format(str(tolerance))),
                                         direction=search_direction, 
                                         allow_exact_matches=True, 
                                         suffixes=('_left', '_right'))
                         )
                
                # Store matches
                matched_results = pd.concat([matches.loc[matches[right_by].notnull(), :], matched_results], ignore_index=False, sort=False)
                
                # Remove matched records for next iteration;
                left = matches[left.columns].loc[matches[right_by].isnull(), :].copy()
                right = right[right.columns].loc[right[right_by].isin(left.opp_Id.unique()), :].copy() 
                matches = pd.DataFrame()
                self.results['merge_asof_results_backward'].add('{0},{1},{2},{3}'.format(tolerance, len(matched_results), len(left), len(right)))
                
                # For unmatched records that remain, use the 'forward' search direction. 
                # Perhaps this should be implemented with remaining unmatched Cases but only after search using 'nearest' and 'backfward' is complete; This approach may match Opp History that was after the S&SE Case was created.
                # Can implement this recursively later; lazy for now...
               
            if len(left) !=0:
                max_days = (left.index.max() - left.index.min()).days
                max_tolerance = list(range(1, max_days))
                for tolerance in max_tolerance:
                    search_direction = 'forward'
                    matches = (pd.merge_asof(left, right, 
                                     left_index=True,  right_index=True, 
                                     left_by=left_by, right_by=right_by, 
                                     tolerance=pd.Timedelta('{0}d'.format(str(tolerance))), direction=search_direction, allow_exact_matches=True, 
                                     suffixes=('_left', '_right'))
                     )

                    # Store matches
                    matched_results = pd.concat([matches.loc[matches[right_by].notnull(), :], matched_results], ignore_index=False, sort=False)
                    # Remove matched records for next iteration
                    left = matches[left.columns].loc[matches[right_by].isnull(), :].copy()
                    right = right[right.columns].loc[right[right_by].isin(left.opp_Id.unique()), :].copy() 
                    matches = pd.DataFrame()
                    self.results['merge_asof_results_forward'].add('{0},{1},{2},{3}'.format(tolerance, len(matched_results), len(left), len(right)))
            
                # For unmatched records that remain, use the 'backward' search direction. 
                # Perhaps this should be implemented with remaining unmatched Cases but only after search using 'nearest' and 'backfward' is complete; This approach may match Opp History that was after the S&SE Case was created.
                # Can implement this recursively later; lazy for now...
            
            if len(left) !=0:
                max_days = (left.index.max() - left.index.min()).days
                max_tolerance = list(range(1, max_days))
                for tolerance in max_tolerance:
                    search_direction = 'backward'
                    matches = (pd.merge_asof(left, right, 
                                     left_index=True,  right_index=True, 
                                     left_by=left_by, right_by=right_by, 
                                     tolerance=pd.Timedelta('{0}d'.format(str(tolerance))), direction=search_direction, allow_exact_matches=True, 
                                     suffixes=('_left', '_right'))
                     )

                    # Store matches
                    matched_results = pd.concat([matches.loc[matches[right_by].notnull(), :], matched_results], ignore_index=False, sort=False)
                    # Remove matched records for next iteration
                    left = matches[left.columns].loc[matches[right_by].isnull(), :].copy()
                    right = right[right.columns].loc[right[right_by].isin(left.opp_Id.unique()), :].copy() 
                    matches = pd.DataFrame()
                    self.results['merge_asof_results_forward'].add('{0},{1},{2},{3}'.format(tolerance, len(matched_results), len(left), len(right)))

            matched_results.sort_index(inplace=True)
            unmatched_results = left.copy()
            
            sys.stdout.write('\n\n{0} matched_results {1} \n\n'.format('\x1b[6;30;43m', '\x1b[0m'))
            sys.stdout.write('{0}'.format(matched_results.info()))
            sys.stdout.write('\n\n{0} unmatched_results {1} \n\n'.format('\x1b[6;30;43m', '\x1b[0m'))
            sys.stdout.write('{0}\n\n'.format(unmatched_results.info()))
            
            del left, right, matches
            return (matched_results, unmatched_results)
        
        except Exception as error:
            sys.stdout.write('\n {0} *** Error: <sf_Manager.match_opp_stage_to_cases()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))
            return pd.DataFrame()
        
    def export_to_csv(self, data, **kwargs):
        
        '''
        Exports Pandas DataFrame to CSV:
        
        Parameters
            data: Pandas DataFrame object to export
            kwargs (dict{}): export parameters: {file_path: 'file_path', file_name: 'file_name', years: list(years)}
        
        Returns:
            
            Export Pandas DataFrame Object to CSV utf-8 encoded file
        '''        
        
        try:
            file_path = GOOGLE_PATH + 'Shareddrives/UB/Sales/Sales & Solution Engineers/Sales/Analysis/data/opps_cases/{0}.csv'.format(kwargs.get('file_path'), kwargs.get('file_name'))
            sys.stdout.write('\n{0} >> Exporting {1} {2}'.format('\x1b[1;33;40m','\x1b[0m',  file_path))
            data.to_csv(path_or_buf=file_path, sep=',', encoding='utf-8')
            
        except Exception as error:
            sys.stdout.write('\n {0} *** Error in <sf_Manager.export_to_csv()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))
            return pd.DataFrame()
        
    def export_to_parquet(self, data, fname='my-parquet-data_{0}.parquet.gzip'.format(pd.Timestamp(datetime.now()).value)):
        
        '''
        Exports Pandas DataFrame to CSV:
        
        Parameters
            data: Pandas DataFrame object to export
            fname (string): file name to export
        
        Returns:
            
            Export Pandas DataFrame Object to CSV utf-8 encoded file
        '''        
        
        try:
        
            # Export Data to Google Drive, as parquet GZip file
            fpath = GOOGLE_PATH + 'Shareddrives/UB/Sales/Sales & Solution Engineers/Sales/Analysis/data/opps_cases/' + fname
            data.to_parquet(path=fpath, index=True)
            sys.stdout.write('\n\n -*- {0}Exported Parquet File: {2}{1} -*- \n\n'.format('\x1b[6;30;46m','\x1b[0m', fpath))
            
        except Exception as error:
            sys.stdout.write('\n {0} *** Error in <sf_Manager.export_to_parquet()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))
            return pd.DataFrame()            

    def read_from_parquet(self, fname='acct_opps.parquet.gzip'):
        
        '''
        Import Parquet File into Pandas DataFrame to CSV:
        
        Parameters
            fname (string): file name to import
        
        Returns:
            Pandas DataFrame Object
        '''        
        try:
            # Export Data to Google Drive, as parquet GZip file
            fpath = GOOGLE_PATH + 'Shareddrives/UB/Sales/Sales & Solution Engineers/Sales/Analysis/data/' + fname
            sys.stdout.write('\n\n -*- {0}Reading Parquet File: {2}{1} -*- \n\n'.format('\x1b[6;30;46m','\x1b[0m', fpath))
            data = pd.read_parquet(path=fpath)
            return data
        except Exception as error:
            sys.stdout.write('\n {0} *** Error in <sf_Manager.read_from_parquet()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))
            return pd.DataFrame()                  

    def _open_json_file(self, path_or_buf=str(os.path.expanduser("~"))):
        
        '''
        Open json file and return results
        
        Parameters:
            path_or_buf (string): Path to JSON File
        
        Returns:
            JSON Dict
        '''      
        try:
            with open(path_or_buf) as json_file:
                data = json.load(json_file)
            return data

        except Exception as error:
            sys.stdout.write('\n {0} *** Error in <sf_Manager._open_json_file()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))
            return 0
    
    def _mount_google_drive(self):
        '''
        Desscription: Mount Google Drive
        
        Parameters:
            None
        
        Returns:
            None
        '''  
        try:
          # Mount Google Drive
          drive.mount('/content/gdrive', force_remount=True)
          # List files for specific Google Drive Directory
          # !ls '/content/gdrive/MyDrive'   
        
        except Exception as error:
            sys.stdout.write('\n {0} *** Error in <sf_Manager.mount_google_drive()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))
            return 0 
         
if __name__ == "__main__":
    
    try:
        
        # Account fields to retrieve; DO NOT use Wilcard (*) with SalesForce SOQL Queries; it does not work; must specify fields in SOQL Query
        acct_labels = 'Id, Name, Type, BillingCountry, Industry, AnnualRevenue, NumberOfEmployees, ' + \
        'CurrencyIsoCode, Account_Owner_Text__c, Total_Opportunities__c, No_of_Open_Optys__c, Region_New__c, ' + \
        'Region__c, Sub_Region__c, Region_Detail__c, Region_text__c, Vertical__c, Business_Segment__c, ZoomInfo_Industry__c, Earliest_Renewal_Date__c, CSM_Role__c, ' + \
        'ProductType_f__c, Current_LMS__c, Account_Record_Type_Name__c, Employee_Segment__c, Customer_Tier__c, LastActivityDate, fcio1__Territory__c, Partner_Type__c'
        
        # Opportunity fields to retrieve;
        opp_labels = 'AccountId, FiscalQuarter, FiscalYear, Id, Name, CreatedDate, CloseDate, Type, ForecastCategory, ForecastCategoryName, CSM_CSA_Oppty_User_ID__c, '+\
        'StageName, Amount, Annual_Recurring_Revenue_ARR__c, RenewalARR__c, Delta_ARR_NEW__c, NewARR__c, New_Bookings_Local_Currency__c, SBCF_Prior_ARR__c, '+\
        'SBCF_Current_ARR__c, JWT_Gross_ARR_Reseller__c, Prior_ARR_Amendment__c, Prior_ARR_Override__c, Opportunity_Open_ARR__c, '+\
        'License_Start_Date_New__c, License_End_Date_New__c, License_Term__c, Contract_Length__c, Contract_Length_CPQ__c, Current_ARR_Override__c, '+\
        'Delta_ARR__c, SBCF_Current_License_Qty__c, SBCF_Prior_License_Qty__c, SBCF_Delta_License_Qty__c, Live_UFB_Owner_String__c, OwnerId, '+\
        'CS_Segment__c, Closed_Lost_Details__c, Closed_Lost_Reason__c, Lost_Reason__c, IsWon, Probability, CurrencyIsoCode, CorpU__c, '+\
        'Opportunity_Name_Append__c, Next_Steps_UFB__c, Recommended_Next_Steps__c, Next_Step_Last_Updated__c, RecordTypeId, NextStep, '+\
        'Current_Content_Provider__c, Dc_Business_Commercial__c, Dc_Decision_Criteria_Technical__c, Dp_Business_Decision_Making__c, Qualified_to_Closed_Days__c, '+\
        'Dp_Technical_Decision_Making__c, E_Economic_Buyer__c, Exec_Sponsor_Engagement__c, Executive_Sponsor__c, I_Indentify_pain__c, '+\
        'I_Identify_Pain_notes__c, M_Metrics_Desired_Outcomes_Notes__c, P_Paper_Process__c, Opportunity_Owner_ID__c, Opportunity_Owner_Role__c, Opportunity_Owner_Account_Owner_Match__c, '+\
        'UFB_Sales_Owner_String_Stamp__c, UFB_Sales_Owner_Segment__c, UFB_Sales_Owner_Region__c, UFB_Sales_Owner_Title__c, UFB_Sales_Owner_Team__c, Product_Type__c, Product_Identification_Pipeline__c'
           
        # Get SalesForce API handle
        sf = sf_Manager(acct_labels=acct_labels, opp_labels=opp_labels) 
    except Exception as error:
        sys.stdout.write('\n{0} **** Error in <sf_manager()__main__> {2} ***** {1} \n\n {3}'.format('\x1b[0;31;40m', '\x1b[0m', str(error), traceback.format_exc()))      

Mounted at /content/gdrive

2023-04-28 16:12:57.786760: Connected to https://udemy.my.salesforce.com/services/data/v52.0/

2023-04-28 16:13:07.654160: Connected to https://udemy.my.salesforce.com/services/data/v52.0/

Querying SOQL Data
SOQL Query: 
SELECT Id, Name FROM User WHERE email IN ('daniel.mcateer@udemy.com', 'garry.hickey@udemy.com', 'rui.calheiros@udemy.com', 'seth.klein@udemy.com', 'steven.reynolds@udemy.com', 'vamsi.pandari@udemy.com', 'ayse.ciftci@udemy.com', 'david.vaughan@udemy.com', 'ryan.petigura@udemy.com', 'joao.carlos@udemy.com', 'erin.ohern@udemy.com', 'paul.sleeman@udemy.com', 'sawandi.cassell@udemy.com', 'jenica.draney@udemy.com', 'christopher.mcdonald@udemy.com', 'dan.fortner@udemy.com', 'kei.pan@udemy.com', 'evelyn.pan@udemy.com', 'william.holton@udemy.com', 'james.caudery@udemy.com', 'kuldeep.harsh@udemy.com', 'lynsie.zellmer@udemy.com', 'sarah.sparks@udemy.com')


#### Get API Data - Clean and Merged

In [ ]:
%%time
%%capture
# Get Accounts and Opps from SFDC API
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
accounts, opportunities = sf.get_accounts_opportunities(years=years)
opportunities_oringinal_rows = opportunities.shape[0]
opportunities.dropna(subset=['opp_Account'],inplace=True)
(sys.stdout.write('\nDropped {0} records with null opp_Account entries\n'
                  .format(opportunities_oringinal_rows - opportunities.shape[0])))

# Get S&SE Cases
soql_query = "SELECT " + sf.case_fields + \
             " FROM Case WHERE OwnerId IN (" + sf.SSEIDs + \
             ") AND RecordTypeName__c IN ('UFB SE Request', 'UFB Rev Ops', 'UB Request for Proposal (RFP)')"
cases = sf.get_cases(soql_query=soql_query)
cases_oringinal_rows = cases.shape[0]
cases.dropna(subset=['case_Related_Opportunity__c'], inplace=True)
sys.stdout.write('\nDropped {0} records with null case_Related_Opportunity__c entries\n'.format(cases_oringinal_rows - cases.shape[0]))

(sys.stdout.write('\nData Stats:\n\n{0}>>accounts: {1}\n>>opportunities:{2}\n>>cases:{3}{4}\n'
                  .format('\x1b[5;30;46m', len(accounts), len(opportunities), len(cases), '\x1b[0m')))

(sys.stdout.write('\n-----{0} Merging Accounts, Opportunities and Cases data  {1}-----\n'
                  .format('\x1b[6;30;43m', '\x1b[0m')))

# Merge Data
acct_opps = sf.get_clean_merged_data(accounts.copy(), opportunities.copy(), cases.copy())
acct_opps.sort_index(axis=0, sort_remaining=True, inplace=True)
(sys.stdout.write('\n\n-----{0} Is Acct Opps Index Sorted? {1} {2}-----\n\n'
                  .format('\x1b[6;30;43m', acct_opps.index.is_monotonic_increasing, '\x1b[0m')))

CPU times: user 2min 41s, sys: 17.4 s, total: 2min 58s
Wall time: 13min 13s


##### Get Data - Parquet File [GDrive]

In [3]:
# Get acct_opps and case_opps Data from GDrive Parquet file
acct_opps = sf.read_from_parquet(fname='acct_opps.parquet.gzip')
# case_opps = (acct_opps.loc[(acct_opps.case_Id.notnull()), ['opp_Id', 'case_Id', 'case_CreatedDate']].copy())



 -*- Reading Parquet File: /content/gdrive/Shareddrives/UB/Sales/Sales & Solution Engineers/Sales/Analysis/data/acct_opps.parquet.gzip -*- 



In [ ]:
acct_opps[acct_opps.opp_Id=='0061T00000wNNLBQA4'].T

##### Get Opp History Merge

In [ ]:
# Get Accout and Opps with S&SE Case Opps Only
left=acct_opps.loc[acct_opps.case_Id.notnull()].copy()
# Pull Opportunity History from SFDC API
# opp_hist=sf.get_opportunity_history()
# Pull Opportunity History from Google Drive Parquet File
opp_hist=sf.read_from_parquet(fname='opp_history.parquet.gzip')
right=opp_hist[['opphist_OpportunityId', 'opphist_CreatedDate_', 'opphist_StageName', 'opphist_ExpectedRevenue']].copy()
right=right.loc[right.opphist_OpportunityId.isin(acct_opps.opp_Id.unique())].copy()

acct_opps_temp=(sf.get_merged_opp_case_history_stages(
    left_data=left.copy(), 
    right_data=right.copy(), 
    right_by='opphist_OpportunityId'))

# Clean fields with NaN
missing_values = {
                'opphist_ExpectedRevenue': 0,
                'opphist_StageName': 'none'
              }
acct_opps_temp.fillna(value=missing_values, inplace=True)
# Set Categorical labels for better performance
categorical_labels = ['opphist_StageName']
acct_opps_temp[categorical_labels] = acct_opps_temp[categorical_labels].apply(pd.Categorical)
# Cast DataFrame objects for better performance before indexing 
cast_cols = {'opphist_ExpectedRevenue': 'float32'}
acct_opps_temp = acct_opps_temp.astype(cast_cols)

# -*- Merge with account_opps -*-
acct_opps_merged = acct_opps.merge(acct_opps_temp[['opphist_CreatedDate_', 'opphist_StageName', 'opphist_ExpectedRevenue']], how='left', left_index=True, right_index=True)
acct_opps_merged.drop_duplicates(subset=['opp_Id', 'case_Id'], inplace=True)
acct_opps_merged.sort_index(axis=1, ascending=True, inplace=True)

del acct_opps
acct_opps = acct_opps_merged.copy()
sf.export_to_parquet(data=acct_opps_merged, fname='acct_opps.parquet.gzip')
del acct_opps_merged, acct_opps_temp, opp_hist, left, right

In [ ]:
acct_opps[['opp_Id', 'case_Id', 'case_CreatedDate', 'opphist_CreatedDate_', 'opphist_StageName', 'opphist_ExpectedRevenue']].loc[acct_opps.opp_Id=='0061T000015aml1QAA']

### Data Analysis

In [4]:
class Data_Analysis():
    
    def __init__(self, **kwargs):
        
        '''
        Description: Various Analysis methods of SalesForce Accounts, Opportunities, and related Cases. 
        The outputs are mostly Pandas DataFrame Objects, such as combined PivotTables, GroupBy DataFrames and such. 
        It also includes key outputs on the S&SE Impact to New Business and Renewal Business, plus other Utilities for Displaying the information via Jupyter or Colab. 
        The methods defined here can all be translated to a Mode Dashboard with the corresponding SQL (or native Python on Mode) providing the same Analytics and Output, but more Scalable.
        
            Parameters:
                kwargs:  parameters to apply filters on Data 
                    *acct_opps (Pandas DataFrame object): aggs_ Account and Opportunities DataFrames
                    *case_opps (Pandas DataFrame object): aggs_ Account, Opportunities and Cases data
                    *is_opp_FiscalYears list([int]): Opp Fiscal Year values to filter DataFrames
                    *is_opp_FiscalQuarters list([int]): Opp Fiscal Year values to filter DataFrames
        '''  
        
        # Select Data as Pandas DataFrame Objects
        self.kwargs = kwargs
        self.acct_opps = self.kwargs.get('acct_opps') # Pandas DataFrame Object: Salesforce Accounts and Opportunity Records
        # self.case_opps = self.kwargs.get('case_opps') # Pandas DataFrame Object: SalesForce Cases
        self.groupby_levels = self.kwargs.get('groupby_levels') # Pandas DataFrame Object: SalesForce Cases
        
        # SalesForce Data Dictionary
        self.data_dictionary = {'opp_ForecastCategories': 
                                [
                                     {'commit': 'Most likely/Probable', 
                                      'best case': 'optimistic', 
                                      'pipeline': 'total opportunity value open in SFDC'
                                     }
                                 ]
                                }
        
        self.agg_dictionary = {'index_': list(self.acct_opps.index.names),
                               'acct_fields': ['acct_Id', 'acct_Employee_Segment__c', 'acct_Account_Owner_Text__c'],
                               'opp_fields': ['opp_Id', 'opp_FiscalYear', 'opp_StageName', 'opp_ForecastCategory', 'opp_Closed_Lost_Details__c', 'opp_Lost_Reason__c', 
                                              'opp_Closed_Lost_Reason__c', 'opp_Closed_Lost_Details__c', 'opphist_StageName', 'opp_SE_Assigned', 'opp_Contract_Length_CPQ__c_Years'],
                               'opp_arr_fields': ['opp_NewARR__c_Converted__USD', 'opp_RenewalARR__c_Converted__USD', 'case_SSE_Percentage_Impact', 
                                                  'opp_NewARR__c_Converted__USD_SSE_IMPACT', 'opp_RenewalARR__c_Converted__USD_SSE_IMPACT'],
                               'case_fields': ['case_Id', 'case_SSEOwnerName', 'case_Creator_Team', 'case_Type', 'case_Sub_Type__c']
                               }

        # Initialize default filter parameters
        self.is_opp_FiscalYears = self.kwargs.get('is_opp_FiscalYears')
        self.is_opp_FiscalQuarters = self.kwargs.get('is_opp_FiscalQuarters')
        self.is_acct_Region_New__c = self.kwargs.get('is_acct_Region_New__c')
        self.current_year = [pd.Timestamp(datetime.now()).year]
        self.current_quarter = [pd.Timestamp(datetime.now()).quarter]
        self.is_opp_Stages =list(self.acct_opps.index.get_level_values(list(self.acct_opps.index.names).index('opp_StageName')).unique())
        self.opp_Stages_Ordered= sorted(list(self.acct_opps.index.get_level_values(list(self.acct_opps.index.names).index('opp_StageName')).unique()))
        self.is_opp_Stages_Open = list([element for element in sorted(list(self.acct_opps.index.get_level_values(2).unique())) if element not in ['Closed Won', 'Closed Lost', 'Closed']])
        self.is_opp_Types = ['New Business', 'Upsell', 'Upgrade', 'Expansion', 'Renewal with Upsell', 'Renewal']
        
        # Color Range for Charts: using Seaborn libary
        self.cm = sns.color_palette("viridis", as_cmap=True)
        
        # Store DataFrames in Dictionary
        self._data_ = {
                            'opps_closed_won_lost': self.get_opps_closed_won_lost(), 
                            'sse_rev_impact': self.get_sse_rev_impact(), 
                            'sse_rev_impact_analysis': self.get_sse_rev_impact_analysis()[0],
                            'sse_rev_impact_analysis_styled': self.get_sse_rev_impact_analysis()[1],
                            'get_opp_narr_stats': self.get_opp_narr_stats(),
                            'get_corpU_opp_stats': self.get_corpU_opp_stats(),
                            'team_plan_opps_won_styled': self.get_team_plan_opps_won()
                    }
        
    def get_opp_narr_stats(self, is_years=[pd.Timestamp(datetime.now()).year], is_quarters=[pd.Timestamp(datetime.now()).quarter], 
                           group_levels=[0, 1], is_opp_region=slice(None), is_opp_type=['New Business', 'Upsell', 'Upgrade', 'Expansion', 'Renewal with Upsell', 'Clawback'], 
                           is_opp_stages=['Closed Won'], is_acct_record_type=['UFB'], is_opp_product_type=['Team Plan'], is_gte_arr=0):
        '''
        Returns a Pandas GroupBy DataFrame object for Account Opps, with S&SE Cases'

        Parameters:
            
            is_years(list([int])): Opp Fiscal Years to filter data in DataFrame; defaults to current Year
            is_quarters(list([int])): Opp Fiscal Quarters to filter data in DataFrame; defaults to current Quarter
            is_opp_region(list([str])): Opp Regions to filter data in DataFrame; defaults to None
            is_opp_type(list([str])): Opp Type to filter data in DataFrame; defaults to None
            is_opp_stages(list([str])): Opp Stages to filter data in DataFrame;  defaults to None
            is_acct_record_type(list([str])): Opp Type to filter data in DataFrame; defaults to None
            is_opp_product_type(list([str])): Opp Type to filter data in DataFrame; defaults to None
            is_gte_arr(int): Opp New ARR filter; default to zero (0)

        Returns: 
                df_grp (Pandas GroupBy DataFrame): GroupBy DataFrame of New ARR Opps (i.e., acct_opps) and Contract Length Bins
        '''
        
        try:
            
            # Get Data
            df = pd.DataFrame()
            df = self.acct_opps.loc[idx[is_years, is_quarters, is_opp_stages, is_opp_type, is_opp_region, :, :, :, :], :].copy()
            df = (df.loc[(df.acct_Account_Record_Type_Name__c.isin(is_acct_record_type))
                         & (~df.opp_Product_Type__c.isin(is_opp_product_type)) 
                         & (~df.opp_Opportunity_Owner_Role__c.isin(['UFB Sales Team Plan - Mgr'])) # Exclude Team Plan Opps
                         # & (df.opp_NewARR__c_Converted__USD >= is_gte_arr)
                        ].copy()
                 )
            
            # Drop Duplicate Opps; created when merging Cases with Opps. 
            df.drop_duplicates(subset=['opp_Id', 'opp_NewARR__c_Converted__USD', 'opp_RenewalARR__c_Converted__USD'], inplace=True)
            df.dropna(subset=['case_Id']) # Also Drop records with no S&SE Cases; i.e., case_id null/nan
            df.sort_index(inplace=True) # Better Pandas performace

            # Build opp_NewARR Bins: there may be a better way. . . lazy approach for now
            bins = ['opp_ARR_lt_50K', 'opp_ARR_gt50K_lt100K', 'opp_ARR_gte_100K', 'opp_ARR_gte_200K', 'opp_ARR_gte_300K', 'opp_ARR_gte_400K',  'opp_ARR_gte_500K', 
                    'opp_Contract_Length_CPQ__c_Year1', 'opp_Contract_Length_CPQ__c_Year2', 'opp_Contract_Length_CPQ__c_Year3']
            df[bins] = 0
            # These are the filters for each bin
            is_lt_50K = (df.opp_NewARR__c_Converted__USD <= 50000)
            is_btw_50_100K = ((df.opp_NewARR__c_Converted__USD > 50000) & (df.opp_NewARR__c_Converted__USD < 100000))
            is_gte_100K = df.opp_NewARR__c_Converted__USD >= 100000
            is_gte_200K = df.opp_NewARR__c_Converted__USD >= 200000
            is_gte_300K = df.opp_NewARR__c_Converted__USD >= 300000
            is_gte_400K = df.opp_NewARR__c_Converted__USD >= 400000
            is_gte_500K = df.opp_NewARR__c_Converted__USD >= 500000

            # Assign NARR to Bins
            df.loc[is_lt_50K, ['{0}'.format(bins[0])]] = df.opp_NewARR__c_Converted__USD.loc[is_lt_50K]
            df.loc[is_btw_50_100K, ['{0}'.format(bins[1])]] = df.opp_NewARR__c_Converted__USD.loc[is_btw_50_100K]
            df.loc[is_gte_100K, ['{0}'.format(bins[2])]] = df.opp_NewARR__c_Converted__USD.loc[is_gte_100K]
            df.loc[is_gte_200K, ['{0}'.format(bins[3])]] = df.opp_NewARR__c_Converted__USD.loc[is_gte_200K]
            df.loc[is_gte_300K, ['{0}'.format(bins[4])]] = df.opp_NewARR__c_Converted__USD.loc[is_gte_300K]
            df.loc[is_gte_400K, ['{0}'.format(bins[5])]] = df.opp_NewARR__c_Converted__USD.loc[is_gte_400K]
            df.loc[is_gte_500K, ['{0}'.format(bins[6])]] = df.opp_NewARR__c_Converted__USD.loc[is_gte_500K]

            # Add Contract Length Bins                   
            df.loc[((df.opp_Contract_Length_CPQ__c >0) & (df.opp_Contract_Length_CPQ__c <=18)), ['{0}'.format(bins[7])]] = 1
            df.loc[((df.opp_Contract_Length_CPQ__c >18) & (df.opp_Contract_Length_CPQ__c <=30)), ['{0}'.format(bins[8])]] = 2
            df.loc[((df.opp_Contract_Length_CPQ__c >30) & (df.opp_Contract_Length_CPQ__c <=100)), ['{0}'.format(bins[9])]] = 3

            # -*- Define Pandas GroupBy Aggs -*-
            aggs = {'{0}'.format(bins[0]): [np.sum],
                    '{0}'.format(bins[1]): [np.sum],
                    '{0}'.format(bins[2]): [np.sum],
                    '{0}'.format(bins[3]): [np.sum],
                    '{0}'.format(bins[4]): [np.sum],
                    '{0}'.format(bins[5]): [np.sum],
                    '{0}'.format(bins[6]): [np.sum],
                    'opp_NewARR__c_Converted__USD': [np.sum],
                    '{0}'.format(bins[7]): [np.count_nonzero],
                    '{0}'.format(bins[8]): [np.count_nonzero],
                    '{0}'.format(bins[9]): [np.count_nonzero],
                   }
            group = (df.groupby(level=[0, 1]).agg(aggs))

            # Flatten GroupBy MultiIndex Column
            group.columns = ['_'.join(col).strip() for col in group.columns.values]
            df.reset_index(inplace=True)
            caption=('Year(s): {0}\nQuarter(s): {1}:\nOpp Stage(s):{2}\nOpp Type(s): {3}\nNo of Opps: {4}\nTotal New ARR: ${5:,.0f}'
                     .format(sorted(df.opp_FiscalYear.unique()), sorted(df.opp_FiscalQuarter.unique()), sorted(df.opp_StageName.unique()), 
                             sorted(df.opp_Type.unique()), len(df.opp_Id.unique()), df.drop_duplicates(subset=['opp_Id']).opp_NewARR__c_Converted__USD.sum()
                            )
                    )
            del df
            return group
        
        except Exception as error:
            sys.stdout.write('\n {0} *** Error in <Data_Analysis.get_opp_narr_stats()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))
            return pd.DataFrame()
            
    def get_opps_closed_won_lost(self):
        '''
        Returns Pandas DataFrame object of SalesForce Opportunities Won and Lost
        
        Parameters:
            None
            
        Returns: 
            DataFrame object of SalesForce Opportunities Won and Lost
        '''
        try:
            # -*- Get Opps WON -*-
  
            df = (self.acct_opps.loc[idx[self.is_opp_FiscalYears, 
                                         self.is_opp_FiscalQuarters, 
                                         ['Closed Won'], 
                                         self.is_opp_Types, 
                                         self.is_acct_Region_New__c, 
                                         :, 
                                         :, 
                                         :, 
                                         :], 
                                     :]
                  .copy()
                  )
            
            # Only UB Opportunities; Exclude NV and Team Plan Opportunities
            df = (df.loc[(df.acct_Account_Record_Type_Name__c.isin(['UFB', 'UFG'])) 
                         & (~df.opp_Product_Type__c.isin(['Team Plan'])) 
                         & (~df.opp_Opportunity_Owner_Role__c.isin(['UFB Sales Team Plan - Mgr']))
                         ].copy()
                  )
            
            # Faster get/setting via sorted index
            df.sort_index(inplace=True)
            df.drop_duplicates(subset=['opp_Id'], inplace=True)
            
            # Groupby DataFrame
            group1 = df.groupby(level=self.groupby_levels).agg(agg1=('opp_Id', 'nunique'),
                                                               agg2=('opp_NewARR__c_Converted__USD', np.sum),
                                                               agg3=('opp_RenewalARR__c_Converted__USD', np.sum)
                                                              )

            # -*- Get Opps LOST -*-
            del df
            df = (self.acct_opps.loc[idx[self.is_opp_FiscalYears, 
                                         self.is_opp_FiscalQuarters, 
                                         ['Closed Lost'], 
                                         self.is_opp_Types, 
                                         self.is_acct_Region_New__c, 
                                         :, 
                                         :, 
                                         :, 
                                         :], 
                                     :]
                  .copy()
                  )
            df = (df.loc[(df.acct_Account_Record_Type_Name__c.isin(['UFB', 'UFG'])) 
                          & (~df.opp_Product_Type__c.isin(['Team Plan'])) 
                          & (~df.opp_Opportunity_Owner_Role__c.isin(['UFB Sales Team Plan - Mgr']))
                          ].copy()
                  )

            # Better performance for getting/setting via sorted index
            df.sort_index(inplace=True)
            df.drop_duplicates(subset=['opp_Id'], inplace=True)

            # Groupby DataFrame
            group2 = (df.groupby(level=self.groupby_levels).agg(agg1_=('opp_Id', 'nunique'),
                                                                agg2_=('opp_NewARR__c_Converted__USD', np.sum),
                                                                agg3_=('opp_RenewalARR__c_Converted__USD', np.sum)
                                                               )
                      )
            
            group1.sort_index(axis=0, ascending=True, inplace=True)
            group2.sort_index(axis=0, ascending=True, inplace=True)
            aggs_ = pd.merge(group1, group2[['agg1_', 'agg2_', 'agg3_']], left_index=True, right_index=True, copy=True)
            rename_mapper = {'agg1': 'no_of_opps_won', 'agg2': 'NARR_C', 'agg3': 'RARR_C', 
                             'agg1_': 'no_of_opps_lost', 'agg2_': 'NARR_C_Lost', 'agg3_': 'RARR_C_Lost'}    
            aggs_.rename(columns=rename_mapper, inplace=True)
            
            del df, group1, group2
            return aggs_
        
        except Exception as error:
            sys.stdout.write('\n {0} *** Error in <Data_Analysis.get_opps_closed_won_lost()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))
            return pd.DataFrame() 
        
    def get_team_plan_opps_won(self):
        '''
        Returns Pandas DataFrame GroupBy Object of Closed Won Team Plan Opportunities
        
        Parameters:
            None
        Returns
            df_grp: Pandas DataFrame GroupBy Object of Closed Won Team Plan Opportunities
        '''
        #-*-Team Plan Opps Closed Won-*-
        try:
            # Select only Team Plan Opps Won
            df = self.acct_opps.loc[idx[self.is_opp_FiscalYears, self.is_opp_FiscalQuarters, ['Closed Won'], :, self.is_acct_Region_New__c, :, :, :, :], :].copy()
            df = (df.loc[(df.acct_Account_Record_Type_Name__c.isin(['UFB', 'UFG'])) 
                         & (df.opp_Product_Type__c.isin(['Team Plan'])) 
                         & (df.opp_Opportunity_Owner_Role__c.isin(['UFB Sales Team Plan - Mgr']))].copy()
                 )
           
            # Faster get/setting via sorted index
            df.sort_index(inplace=True)
            df.drop_duplicates(subset=['opp_Id'], inplace=True)

            # Group by OppFiscalYear and OppFiscalQuarter (level's 0 and 1 of the acct_opps data) DataFrame 
            df_grp = df.groupby(level=self.groupby_levels).agg(agg1=('opp_NewARR__c_Converted__USD', np.sum))
            # Flatten Groupby DataFrame                                    
            df_grp.sort_index(axis=0, ascending=True, inplace=True)           
            df_rename_mapper = {
                                'agg1': 'TP_NARR_C'
            }          
            df_grp.rename(columns=df_rename_mapper, inplace=True)
            
            del df
            return df_grp
        
        except Exception as error:
            sys.stdout.write('\n {0} *** Error in <Data_Analysis.get_team_plan_opps_won()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))
            return pd.DataFrame() 
            
    def get_sse_rev_impact(self, is_acct_record_type=['UFB', 'UFG'], is_opp_product_type=['Team Plan']):
        
        '''
            Returns:  S&SE Revenue Impact based on SalesForce Cases opened. Excludes Team Plan,  NV/Reseller Opportunities.  
            
            Parameters:
                None
            
            Returns:
               aggs_ (Pandas GroupBy DataFrame): aggs_ Pandas GroupBy outputs into one DataFrame'. 
        '''
        # -*- SSE Cases with Opps Won and Lost-*-
        try:
            
            # -*- Get Opps WON w/ S&SE Engagement Cases -*-  
            df = self.acct_opps.loc[idx[:, :, :, :, self.is_acct_Region_New__c, :, :, self.is_opp_FiscalYears, self.is_opp_FiscalQuarters], :].copy()
            df = df.loc[(df.case_Id.notnull())].copy()
            
            # If Grouping by ['opp_FiscalYear', 'opp_FiscalQuarter'], then calcuate the total number of S&SE Cases grouped by ['case_CreatedYear', 'case_CreatedQuarter']
            check = all(item in self.groupby_levels for item in ['opp_FiscalYear', 'opp_FiscalQuarter'])
            if check:
              groupby_levels=['case_CreatedYear', 'case_CreatedQuarter']
            else:
              groupby_levels=self.groupby_levels

            number_sse_cases = (df.loc[(df.case_Id.notnull())].groupby(level=groupby_levels)
                                  .agg(number_sse_cases=('case_Id','count')) # Total number of S&SE Cases
                               )
            del df
            
            # -*- Get Opps WON and New ARR and Renewal ARR w/ S&SE Engagement Cases; exclude Team Plan Opps -*-
            df = self.acct_opps.loc[idx[self.is_opp_FiscalYears, self.is_opp_FiscalQuarters, ['Closed Won'], self.is_opp_Types, self.is_acct_Region_New__c, :, :, :, :], :].copy()
            df = (df.loc[((df.case_Id.notnull())
                         & df.acct_Account_Record_Type_Name__c.isin(['UFB', 'UFG'])) 
                         & (~df.opp_Product_Type__c.isin(['Team Plan'])) 
                         & (~df.opp_Opportunity_Owner_Role__c.isin(['UFB Sales Team Plan - Mgr']))].copy()
                 )
            
            # Better performance for getting/setting via sorted index; remove dups records by Opp_Id, and ARR fields
            df.sort_index(inplace=True)
            df.drop_duplicates(subset=['opp_Id', 'opp_NewARR__c_Converted__USD', 'opp_RenewalARR__c_Converted__USD'], inplace=True)

            # Groupby 'Closed Won' Opps DataFrame 
            group1 = df.groupby(level=self.groupby_levels).agg(agg1=('opp_Id', 'nunique'), 
                                                               agg2=('opp_NewARR__c_Converted__USD', np.sum),
                                                               agg3=('opp_RenewalARR__c_Converted__USD', np.sum)
                                                               )
            number_sse_cases.index.names = group1.index.names
            group1 = pd.merge(group1, number_sse_cases, left_index=True, right_index=True, copy=True)
            del df

            # -*- Get Opps LOST and New ARR and Renewal ARR w/ S&SE Engagement Cases -*-
            df = self.acct_opps.loc[idx[self.is_opp_FiscalYears, self.is_opp_FiscalQuarters, ['Closed Lost'], self.is_opp_Types, self.is_acct_Region_New__c, :, :, :, :], :].copy()
            df = (df.loc[(df.acct_Account_Record_Type_Name__c.isin(is_acct_record_type)) 
                         & (df.case_Id.notnull())
                         & (~df.opp_Product_Type__c.isin(['Team Plan'])) 
                         & (~df.opp_Opportunity_Owner_Role__c.isin(['UFB Sales Team Plan - Mgr']))].copy())

            # Better performance for getting/setting via sorted index; remove dups records by Opp_Id, and ARR fields
            df.sort_index(inplace=True)
            df.drop_duplicates(subset=['opp_Id', 'opp_NewARR__c_Converted__USD', 'opp_RenewalARR__c_Converted__USD'], inplace=True)

            # Groupby 'Closed Lost' Opps DataFrame 
            group2 = df.groupby(level=self.groupby_levels).agg(agg1_=('opp_Id', 'nunique'), 
                                                               agg2_=('opp_NewARR__c_Converted__USD', np.sum), 
                                                               agg3_=('opp_RenewalARR__c_Converted__USD', np.sum)
                                                               )
            # Merge Grouped DataFrames
            group1.sort_index(axis=0, ascending=True, inplace=True)
            group2.sort_index(axis=0, ascending=True, inplace=True)
            aggs_ = pd.merge(group2[['agg1_', 'agg2_', 'agg3_']], group1, left_index=True, right_index=True, copy=True)
 
            # Rename Columns
            aggs__rename_mapper = {'agg1': 'no_sse_opps_won', 
                                   'agg2': 'SSE_IMPACT_NARR_C',
                                   'agg3': 'SSE_IMPACT_RARR_C',
                                   'agg1_': 'no_sse_opps_lost',
                                   'agg2_': 'SSE_NARR_C_LOST',
                                   'agg3_': 'SSE_RARR_C_LOST'
                                }
            aggs_.rename(columns=aggs__rename_mapper, inplace=True)

            aggs__label_formatter = {}
            for k, v in aggs__rename_mapper.items():
                if 'ARR' in v:
                    aggs__label_formatter[v] = '${:,.0f}'
                else:
                     aggs__label_formatter[v] = '{:,.0f}'

            # Style output DataFrame
            output_labels = ['no_sse_opps_won', 'number_sse_cases', 'SSE_IMPACT_NARR_C', 'SSE_IMPACT_RARR_C', 'no_sse_opps_lost', 'SSE_NARR_C_LOST', 'SSE_RARR_C_LOST']
            gradient_subset_labels = ['SSE_IMPACT_NARR_C', 'SSE_IMPACT_RARR_C', 'SSE_NARR_C_LOST', 'SSE_RARR_C_LOST'] 
            aggs__styled = (aggs_[output_labels].style.format(aggs__label_formatter).background_gradient(cmap=self.cm, subset=gradient_subset_labels))

            self.sse_rev_impact_grp = aggs__styled
            del df, aggs__styled
            return aggs_
        
        except Exception as error:
            sys.stdout.write('\n {0} *** Error in <Data_Analysis.get_sse_rev_impact()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))
            return pd.DataFrame() 
            
    def get_sse_rev_impact_analysis(self):
        
        '''
        Returns S&SE Team's Revenue Impact on UB Opportunitied; excludes Team Plan, NV/Reseller Opportunities
        
        Parameters:
            None
        
        Returns
            aggs_2 (Pandas GroupBy DataFrame): aggs_ GroupBy DataFrame of S&SE Revenue Impact
            aggs_2__styled (Pandas GroupBy DataFrame Styled): Styled aggs_2 output
        '''
        
        # -*-SSE Revenue Impact on Opps Won-*-
        try:    
            # First, Get Grouped Dataframes for Opps Won/Lost, SSE ARR Impact, and Team Plan ARR
            grp1 = self.get_opps_closed_won_lost()
            grp2 = self.get_sse_rev_impact()
            
            # Now, merge the above Grouped Dataframes
            aggs_1 = pd.merge(grp1, grp2, left_index=True, right_index=True, copy=True)
            # aggs_2.drop(columns=['opp_FiscalYear_x', 'opp_FiscalQuarter_x', 'opp_FiscalYear_y', 'opp_FiscalQuarter_y'], inplace=True)
            del grp1, grp2

            # Rename labels
            aggs_1_rename_mapper = {
                                'current_column_label': 'new_column_label'
            }

            # Calculate SSE Impact Ratio Data
            aggs_1['SSE_IMPACT_NARR_C_RATIO'] = aggs_1.SSE_IMPACT_NARR_C/(aggs_1.NARR_C)
            aggs_1['SSE_IMPACT_RARR_C_RATIO'] = aggs_1.SSE_IMPACT_RARR_C/(aggs_1.RARR_C)
            aggs_1['SSE_IMPACT_NARR_C_LOST_RATIO'] = aggs_1.SSE_NARR_C_LOST/(aggs_1.NARR_C_Lost)  
            aggs_1['SSE_IMPACT_RARR_C_LOST_RATIO'] = aggs_1.SSE_RARR_C_LOST/(aggs_1.RARR_C_Lost)  
            
            # Format label outputs
            aggs_1_format_dict = {
                                    'NARR_C': '${:,.0f}',
                                    'RARR_C': '${:,.0f}',
                                    'NARR_C_Lost': '${:,.0f}',
                                    'RARR_C_Lost': '${:,.0f}',
                                    'SSE_IMPACT_NARR_C': '${:,.0f}',
                                    'SSE_IMPACT_RARR_C': '${:,.0f}',
                                    'SSE_NARR_C_LOST': '${:,.0f}',
                                    'SSE_RARR_C_LOST': '${:,.0f}',
                                    'SSE_IMPACT_NARR_C_RATIO': '{:.2%}',
                                    'SSE_IMPACT_RARR_C_RATIO': '{:.2%}',
                                    'SSE_IMPACT_NARR_C_LOST_RATIO': '{:.2%}',
                                    'SSE_IMPACT_RARR_C_LOST_RATIO': '{:.2%}',
                            }
    
            # Stylize DataFrame Output
            output_labels=[ 
                            'no_of_opps_won', 'no_sse_opps_won', 'no_of_opps_lost', 'no_sse_opps_lost', 'number_sse_cases', 
                            'NARR_C', 'SSE_IMPACT_NARR_C', 'SSE_IMPACT_NARR_C_RATIO', 'RARR_C', 'SSE_IMPACT_RARR_C', 'SSE_IMPACT_RARR_C_RATIO',
                            'NARR_C_Lost', 'SSE_NARR_C_LOST', 'SSE_IMPACT_NARR_C_LOST_RATIO', 'RARR_C_Lost', 'SSE_RARR_C_LOST', 'SSE_IMPACT_RARR_C_LOST_RATIO'
                          ]
            background_gradient_subset=['NARR_C', 'SSE_IMPACT_NARR_C', 'SSE_IMPACT_NARR_C_RATIO', 'RARR_C', 'SSE_IMPACT_RARR_C', 'SSE_IMPACT_RARR_C_RATIO', 
                                        'NARR_C_Lost', 'SSE_NARR_C_LOST', 'SSE_IMPACT_NARR_C_LOST_RATIO', 'RARR_C_Lost', 'SSE_RARR_C_LOST', 'SSE_IMPACT_RARR_C_LOST_RATIO']
            
            # aggs_2.reset_index(drop=False, inplace=True)
            aggs_1__styled = (aggs_1[output_labels].style.format(aggs_1_format_dict)
                                                     .background_gradient(cmap=self.cm, subset=background_gradient_subset)
                                                     .set_caption('S&SE ARR Performace: {0}'.format(aggs_1.index.unique()))
                                                     )

            return aggs_1, aggs_1__styled
        
        except Exception as error:
            sys.stdout.write('\n {0} *** Error in <Data_Analysis.get_sse_rev_impact_analysis()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))
            return pd.DataFrame()       
    
    def get_corpU_opp_stats(self,):
        '''
        Returns Grouped Pandas DataFrames for Learning Academy Opportunities in SalesForce in Current Year
        
        Parameters:
            group_levels (list[int]): Multi-index levels to group by. The DataFrame must be a Multi-Index aligned DataFrame
        
        Returns:
            df_grp (Pandas DataFrame): Grouped Pandas DataFrame of Learning Academy Opportunities
            df_pvt (Pandas DataFrame): PivotTable Pandas DataFrame of Learning Academy Opportunities
        '''
        # -*-CorpU Opps-*-
        try:
            
            # Filter the DataFrame for CorpU Opportunities
            df = self.acct_opps.loc[idx[[pd.Timestamp(datetime.now()).year], :, self.is_opp_Stages_Open, :, self.is_acct_Region_New__c, :, :, :, :], :].copy()
            df = df.loc[((df.opp_CorpU__c) | (df.opp_Opportunity_Name_Append__c.str.contains('CorpU')))].copy()
            
            # Faster get/setting via sorted index; drop Opp dups
            df.sort_index(inplace=True)
            df.drop_duplicates(subset=['opp_Id'], inplace=True)

            # -*- DATA FRAME 1: GroupBy -*-
            df_grp = df.groupby(level=self.groupby_levels).agg(agg1=('opp_Id', np.count_nonzero), 
                                                               agg2=('opp_NewARR__c_Converted__USD', np.sum))

            # Flatten Groupby DataFrame & rename columns
            df_grp.reset_index(inplace=True)
            df_rename_mapper = {
                                'agg1': 'NoOpps', 
                                'agg2': 'NARR'
            }      
            df_grp.rename(columns=df_rename_mapper, inplace=True)

            # Remove missing values (nan)
            fill_na={}
            for column in df_grp.columns:
                if (df_grp[column].dtype) in (np.int64, np.float64):
                    fill_na[column]=0
            df_grp.fillna(fill_na, inplace=True)

            # Define Format Labels
            df_label_formatter = {}  
            df_columns = ['{}'.format(field) for field in list(df_grp.columns)]
            for column in list(df_grp.columns):
                if 'ARR' in column:
                    df_label_formatter[column] = '${:,.0f}'
                else:
                    df_label_formatter[column] = '{}'

            # Style Output DataFrame
            df_grp_styled = (df_grp.style
                             .format(df_label_formatter)
                             .background_gradient(cmap=self.cm, subset=['NoOpps', 'NARR'])
                             .set_caption('CorpU NARR Opps [{}] ${:,.0f}\n'.format([pd.Timestamp(datetime.now()).year], df_grp.NARR.values.sum())
                                         )
                            )

            # -*- DATA FRAME 2: PivotTable -*-
            index=['opp_FiscalQuarter']
            columns=['opp_ForecastCategoryName']
            df.reset_index(inplace=True)
            df_columns_subset=['opp_FiscalQuarter', 'opp_Id', 'opp_NewARR__c_Converted__USD', 'opp_ForecastCategoryName']
            df_pvt = pd.pivot_table(data=df[df_columns_subset], 
                                    values=None, 
                                    index=index, 
                                    columns=columns, 
                                    aggfunc={'opp_NewARR__c_Converted__USD': np.sum,
                                             'opp_Id': np.count_nonzero},
                                    margins=False,
                                    margins_name='Total',
                                    fill_value=0
                                   )

            # Remove missing values (nan)
            fill_na={}
            for column in df_pvt.columns:
                if (df_pvt[column].dtype) in (np.int64, np.float64):
                    fill_na[column]=0
            df_pvt.fillna(fill_na, inplace=True)

            # Output Formatting
            df_label_formatter = {}
            if isinstance(df_pvt.columns, pd.MultiIndex):
                for k, v, in df_pvt.columns:
                    if (('ARR' in k) | ('ARR' in v)):
                        df_label_formatter[(k,v)] = '${:,.0f}'
                    else:
                        df_label_formatter[(k,v)] = '{}'
            else: 
                for column in df_pvt.columns:
                    if 'ARR' in column:
                        df_label_formatter[column] = '${:,.0f}'
                    else:
                        df_label_formatter[column] = '{}'
            
            df_pvt_styled = (df_pvt.style
                             .format(df_label_formatter)
                             .background_gradient(cmap=self.cm)
                             .set_caption('{} CorpU Open Opps NARR Total: ${:,.0f}\n'.format([pd.Timestamp(datetime.now()).year], df_grp.NARR.values.sum()))
                            )

            del df
            return (df_grp, df_grp_styled, df_pvt, df_pvt_styled)
        
        except Exception as error:
            sys.stdout.write('\n {0} *** Error in <Data_Analysis.get_corpU_opp_stats()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))
            return pd.DataFrame()         

    def get_pivot_table_analysis(self, data=pd.DataFrame(), index=list(), columns=list(), values=list(), drop_opp_dups=True, include_total=True, sort_asc=True):

      '''
        Returns multiple Pandas PivoTables for specified Data and Group Parameters
        
        Parameters:
            Data (Pandas Data Object): to apply PivotTable [Required]
            index (list): list to index on in Pandas PivotTable [Required]
            columns (list): list for Column headers in Pandas PivoTable [Required]
            values (list): list for Column Names in DataFrame to create aggregations [Required]
            drop_opp_dups (boolean): list for Column headers in Pandas PivoTable [Required]
        
        Returns
            aggs_2 (Pandas GroupBy DataFrame): aggs_ GroupBy DataFrame of S&SE Revenue Impact
            aggs_2__styled (Pandas GroupBy DataFrame Styled): Styled aggs_2 output
      '''   

      try:
            pvt_pctg1=pd.DataFrame() 
            pvt_pctg2=pd.DataFrame()
            # Better performance for getting/setting via sorted index; drop Opp_Id
            data.sort_index(inplace=True)
            if drop_opp_dups:
              data.drop_duplicates(subset=['opp_Id'], inplace=True)

            # Build Agg for PivoTable from Values passed
            if not values:
              return None
            if values:
              agg_func={}
              for value in values:
                if 'ARR' in value:
                  agg_func[value]=np.sum
                  # agg_func[value]=np.mean
                  # agg_func[value]=np.median
                elif '_Id' in value:
                  agg_func[value]=np.count_nonzero
                else:
                  agg_func[value]=np.count_nonzero
            print(agg_func)
            pvt = (pd.pivot_table(data=data.reset_index(), 
                                  values=values, 
                                  index=index, 
                                  columns=columns,
                                  aggfunc=agg_func, 
                                  margins=include_total,
                                  dropna=False,
                                  fill_value=0
                                  )
                  )
            pvt.sort_values(by=[pvt.columns[-1]], ascending=False, inplace=True)
            
            # Calculate Pctg Change over Grand Total
            if include_total:
              pvt_pctg1 = pd.DataFrame()
              grd_total = pvt.loc[idx['All'], pvt.columns[::-len(pvt.columns)]].values
              arr = ((pvt.values/grd_total))
              pvt_pctg1 = pd.DataFrame(arr, columns=pvt.columns, index=pvt.index)
            
              # Calculate Pctg Change over Row Index Total
              pvt_pctg2 = pd.DataFrame()
              arr = np.nan_to_num(pvt.values/pvt.head(n=1).values)
              pvt_pctg2 = pd.DataFrame(arr, columns=pvt.columns, index=pvt.index)
            
              # Sort by the Last Column in the DataFrame; if margings=True in Pivot Table, sorts by index Total
              if sort_asc:
                pvt_pctg1.sort_values(by=[pvt.columns[-1]], ascending=False, inplace=True)
                pvt_pctg2.sort_values(by=[pvt.columns[-1]], ascending=False, inplace=True)
            
            # pvt.fillna(value=0, inplace=True)
 
            return (pvt, pvt_pctg1, pvt_pctg2)

      except Exception as error:
          sys.stdout.write('\n {0} *** Error in <Data_Analysis.get_pivot_table_analysis()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))
          return pd.DataFrame() 
    
    def style_data_frame(self, data=pd.DataFrame(), pct_format=False):
        '''
        Returns Styled Pandas DataFrame Object 
        
            Parameters:
                data: Pandas DataFrame Object
            
            Returns:
                data_styled (Pandas DataFrame Styled Object): Pandas DataFrame GroupBy Styled Object
        ''' 
        try:

                # Replace missing values (nan) for int or float columns
                fill_na={}
                for column in data.columns:
                    if (data[column].dtype) in (np.int64, np.float64):
                        fill_na[column]=0
                data.fillna(fill_na, inplace=True)
                
                # Style DataFrame as Currenty or Percentage
                data_label_formatter={}
                if isinstance(data.columns, pd.MultiIndex): 
                  sub = 'ARR'
                  for columns in list(data.columns):
                    for col in list(columns):
                      if (any(s for s in ['{0}'.format(col)] if sub.lower() in s.lower())):
                        data_label_formatter[columns] = '${:,.0f}'
                if pct_format:
                   for columns in list(data.columns):
                     data_label_formatter[columns] = '{:.2%}'
                
                # Tool Tip highlighting
                tt = pd.DataFrame([['-']*len(data.columns)], 
                                  index=['All'], 
                                  columns=data.columns
                                ) 
                last_column = data.columns[::-len(data.columns)]
                cell_border = pd.DataFrame([['border-blue ']*len(data.columns[::-len(data.columns)])], index=['All'], columns=data.columns[::-len(data.columns)])
                data_styled = (data.style.format(formatter=data_label_formatter)
                .background_gradient(cmap=sns.color_palette('rocket_r', as_cmap=True), subset=list(data_label_formatter.keys()))
                .set_tooltips(tt, props='visibility: hidden; position: absolute; z-index: 1; border: 2px solid #242CB1;'
                                        'background-color: white; color: #000066; font-size: 1em;'
                                        'transform: translate(0px, -24px); padding: 0.6em; border-radius: 0.5em;')
                              .set_table_styles(
                                  [
                                      {
                                        'selector': '.border-red', 'props': 'border: 2px dashed red;'
                                        }, 
                                      {'selector': '.border-green', 'props': 'border: 2px dashed green;'
                                        }, 
                                      {'selector': '.border-blue', 'props': 'border: 2px dashed blue;'
                                        }
                                    ], 
                                    overwrite=False)
                              .set_td_classes(cell_border)
                )
            
                return data_styled
        except Exception as error:
            sys.stdout.write('\n {0} *** Error in <Data_Analysis.style_data_frame()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))
            return pd.DataFrame()  

    def chart_missing_data(self, data=pd.DataFrame()):
        
        '''
        Seaboarn Barchart of Missing Data elements in Pandas DataFrame Object
            
            Paramters:
                data (Pandas DataFrame object)
            
            Returns:
                Seaboarn Barchart object
        '''       
        
        try:
            missing_data = data.isna().sum().div(data.shape[0]).mul(100).to_frame().sort_values(by=0, ascending=True)
            fig, ax = plt.subplots(figsize=(30, 10))
            sns.barplot(x=missing_data[missing_data[0]>0].index, y=missing_data[missing_data[0]>0][0])
            plt.ylabel('Pctg Of Missing Values in Data', fontsize=15, fontweight='bold')
            plt.xlabel('Features', fontsize=15, fontweight='bold')
            plt.xticks(rotation=90)

            return plt.show()
       
        except Exception as error:
            sys.stdout.write('\n {0} *** Error in <Data_Analysis.chart_missing_data()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))
            return pd.DataFrame() 
        
    def display(self, data_frames=dict()):
        '''
        Returns Modal of Pandas DataFrame Object
        
        Parameters:
            data_frames (Pandas DataFrame Object): Pandas DataFrame to display in Jupyter Modal Window
        Returns:
            None
        '''
        try:
            # Display Output
            for key in data_frames.keys():
                sc = Sidecar(title=str(key), anchor='split-bottom')
                with sc:
                    display(data_frames[key])
        except Exception as error:
            sys.stdout.write('\n {0} *** Error in <Data_Analysis.display()> *** {1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))
            return 0
                
if __name__ == "__main__":
    try:
        # Seed Data and Filters
        da = Data_Analysis(acct_opps=acct_opps.copy(), is_opp_FiscalYears=[2020, 2021, 2022, 2023], is_opp_FiscalQuarters=[1, 2, 3, 4], is_acct_Region_New__c=All, groupby_levels=['opp_FiscalYear', 'opp_FiscalQuarter'])
        sys.stdout.write('\n{0} . . . Data Analysis'.format(datetime.now()))
    except Exception as error:
        sys.stdout.write('\n{0} *** Error in <Data_Analysis__main__> ***{1}\n{2}'.format('\x1b[0;31;40m', '\x1b[0m', traceback.format_exc()))


2023-04-28 16:13:17.803815 . . . Data Analysis

In [9]:
#@title #### Global S&SE Revenue Performance Metrics { run: "auto" }

# Form Filters
is_opp_FiscalYears_list = All #@param ["All", "[2021]", "[2022]", "[2023]", "[2021, 2022, 2023]"] {type:"raw", allow-input: true}
is_opp_FiscalQuarters_list = All #@param ["All", "[1]", "[2]", "[3]", "[4]"] {type:"raw", allow-input: true}
is_acct_Region_New__c_list = All #@param ["All", "\"APAC\"", "\"AMER\"", "\"EMEA\"", "\"LATAM\"", "\"AMER LATAM\""] {type:"raw", allow-input: true}

# Convert values to List if not a Slice (i.e., not 'All' is selected)
if not isinstance(is_opp_FiscalYears_list, slice):
  is_opp_FiscalYears_list=list(is_opp_FiscalYears_list)
if not isinstance(is_opp_FiscalQuarters_list, slice):
  is_opp_FiscalQuarters_list=list(is_opp_FiscalQuarters_list)
if not isinstance(is_acct_Region_New__c_list, slice):
  is_acct_Region_New__c_list=is_acct_Region_New__c_list.split()

print(is_opp_FiscalYears_list,is_opp_FiscalQuarters_list,is_acct_Region_New__c_list)
button = widgets.Button(description="Run Analysis!")
output = widgets.Output()

def on_button_clicked(b):
  # Display the message within the output widget.
  with output:
    da = Data_Analysis(acct_opps=acct_opps.copy(),
                       is_opp_FiscalYears=is_opp_FiscalYears_list, 
                       is_opp_FiscalQuarters=is_opp_FiscalQuarters_list, 
                       is_acct_Region_New__c=is_acct_Region_New__c_list, 
                       groupby_levels=['opp_FiscalYear', 'opp_FiscalQuarter'])
    da.display(data_frames={'S&SE ARR Impact': da._data_['sse_rev_impact_analysis_styled']})
button.on_click(on_button_clicked)
display(button, output)

slice(None, None, None) slice(None, None, None) slice(None, None, None)


Button(description='Run Analysis!', style=ButtonStyle())

Output()

In [30]:
#@title #### Global S&SE Case Engagement Metrics { run: "auto" }
# Form Filters
is_opp_FiscalYears_list = All #@param ["All", "[2021]", "[2022]", "[2023]", "[2021, 2022, 2023]"] {type:"raw", allow-input: true}
is_opp_FiscalQuarters_list = All #@param ["All", "[1]", "[2]", "[3]", "[4]"] {type:"raw", allow-input: true}
is_acct_Region_New__c_list = All #@param ["All", "\"APAC\"", "\"AMER\"", "\"EMEA\"", "\"LATAM\"", "\"AMER, LATAM\""] {type:"raw", allow-input: true}
is_opp_StageNames_list = All #@param ["All", "\"Closed Won\"", "\"Closed Lost\""] {type:"raw", allow-input: true}
is_opp_Types_list = "ARR Subset" #@param ["All", "ARR Subset"] {allow-input: true}
is_case_CreatedYears_list = [2023] #@param ["All", "[2021]", "[2022]", "[2023]", "[2021, 2022, 2023]"] {type:"raw", allow-input: true}
is_case_CreatedQuarters_list = [1] #@param ["All", "[1]", "[2]", "[3]", "[4]"] {type:"raw", allow-input: true}

# Convert values to List if not a Slice (i.e., not 'All' is selected)
if not isinstance(is_opp_FiscalYears_list, slice):
  is_opp_FiscalYears_list=list(is_opp_FiscalYears_list)
if not isinstance(is_opp_FiscalQuarters_list, slice):
  is_opp_FiscalQuarters_list=list(is_opp_FiscalQuarters_list)
if not isinstance(is_acct_Region_New__c_list, slice):
  is_acct_Region_New__c_list=is_acct_Region_New__c_list.split()
if not isinstance(is_opp_StageNames_list, slice):
  is_opp_StageNames_list=is_opp_StageNames_list.split('  ', maxsplit=1)
if not isinstance(is_case_CreatedYears_list, slice):
  is_case_CreatedYears_list=list(is_case_CreatedYears_list)
if not isinstance(is_case_CreatedYears_list, slice):
  is_case_CreatedQuarters_list=list(is_case_CreatedQuarters_list)
if is_opp_Types_list == 'All':
  is_opp_Types_list=All
if is_opp_Types_list== 'ARR Subset':
  is_opp_Types_list=da.is_opp_Types

print('[{0}, {1}, {2}, {3}, {4}, {5}, {6}]'.format(is_opp_FiscalYears_list, is_opp_FiscalQuarters_list, 
                                                   is_acct_Region_New__c_list, is_opp_StageNames_list, 
                                                   is_opp_Types_list, is_case_CreatedYears_list, 
                                                   is_case_CreatedQuarters_list))

data=acct_opps.loc[idx[is_opp_FiscalYears_list, 
                       is_opp_FiscalQuarters_list, 
                       is_opp_StageNames_list, 
                       is_opp_Types_list, 
                       is_acct_Region_New__c_list, 
                       :, 
                       :, 
                       is_case_CreatedYears_list, 
                       is_case_CreatedQuarters_list], 
                   :].copy()
data=(data.loc[(data.case_Id.notnull())
& (data.acct_Account_Record_Type_Name__c.isin(['UFB', 'UFG']) )
& (~data.opp_Product_Type__c.isin(['Team Plan'])) 
& (~data.opp_Opportunity_Owner_Role__c.isin(['UFB Sales Team Plan - Mgr']))
& (data.loc[data.opp_UFB_Sales_Owner_Segment__c.isin(['ENT'])])
& (data.case_SSEOwnerName.isin(list(sf.sse_assignments['AMER'][0].keys())))
])
if data.empty:
  print('No Data')
  exit()

button = widgets.Button(description="Run Analysis!")
output = widgets.Output()

# {'index_': ['opp_FiscalYear', 
#             'opp_FiscalQuarter',
#             'opp_StageName',
#             'opp_Type',
#             'acct_Region_New__c',
#             'acct_Region__c',
#             'acct_Business_Segment__c',
#             'case_CreatedYear',
#             'case_CreatedQuarter'
#             ],
#  'acct_fields': ['acct_Id', 
#                  'acct_Employee_Segment__c',
#                  'acct_Account_Owner_Text__c'
#                   ],
#  'opp_fields': ['opp_Id', 
#                 'opp_FiscalYear',
#                 'opp_StageName',
#                 'opp_ForecastCategory',
#                 'opp_Closed_Lost_Details__c',
#                 'opp_Lost_Reason__c',
#                 'opp_Closed_Lost_Reason__c',
#                 'opp_Closed_Lost_Details__c',
#                 'opphist_StageName',
#                 'opp_SE_Assigned',
#                 'opp_Contract_Length_CPQ__c_Years',
#                 'opphist_StageName'
#                 ],
#  'opp_arr_fields': ['opp_NewARR__c_Converted__USD', 
#                     'opp_RenewalARR__c_Converted__USD',
#                     'case_SSE_Percentage_Impact',
#                     'opp_NewARR__c_Converted__USD_SSE_IMPACT',
#                     'opp_RenewalARR__c_Converted__USD_SSE_IMPACT'
#                     ],
#  'case_fields': ['case_Id',
#                  'case_SSEOwnerName',
#                  'case_Creator_Team',
#                  'case_Type',
#                  'case_Sub_Type__c'
#                  ]
#  }

def on_button_clicked(b):
  # Display the message within the output widget.
  with output:
    index = ['case_SSEOwnerName']
    columns = ['case_CreatedWeek']
    values = ['case_Id']
    (pvt, pvt_pctg1, pvt_pctg2) = da.get_pivot_table_analysis(data=data, 
                                                              index=index, 
                                                              columns=columns, 
                                                              values=values, 
                                                              drop_opp_dups=False, 
                                                              include_total=True, 
                                                              sort_asc=False)
    (da.display(data_frames={'S&SE ARR Impact': 
                             (da.style_data_frame(data=pvt
                                                  # .loc[:, ('opp_NewARR__c_Converted__USD', 'AMER')]
                                                  # .loc[pvt[('opp_NewARR__c_Converted__USD', 'All')]>0]
                                                  # .sort_index()
                                                  )
                             )
                             }
                )
    )
button.on_click(on_button_clicked)
display(button, output)

[slice(None, None, None), slice(None, None, None), slice(None, None, None), slice(None, None, None), ['New Business', 'Upsell', 'Upgrade', 'Expansion', 'Renewal with Upsell', 'Renewal'], [2023], [1]]


In [ ]:
pvt

opp_NewARR__c_Converted__USD                 \
acct_Employee_Segment__c                    1: 1–199 Emerging 2: 200–999 SMB   
opp_FiscalYear opp_FiscalQuarter                                               
All                                             580881.875000     1821366.50   
2022           1                                260579.171875      937194.25   
2023           1                                320302.718750      884172.25   

                                                            \
acct_Employee_Segment__c         3: 1,000–5,000 Mid Market   
opp_FiscalYear opp_FiscalQuarter                             
All                                             1203008.75   
2022           1                                 817104.75   
2023           1                                 385904.00   

                                                                            
acct_Employee_Segment__c         4: 5,001+ Enterprise 5: Other         All  
opp_FiscalYear opp_FiscalQuarter                                            
All                                        8327632.00        0  11932890.0  
2022           1                           5280566.50        0   7295445.0  
2023           1                           3047065.25        0   4637444.0

In [ ]:
arr1 = pvt.loc[idx[2022, 1], :].values[:-1]/pvt.loc[idx[2022, 1], :].values[-1]
arr1

array([0.03571806, 0.12846293, 0.11200204, 0.72381692, 0.        ])

In [ ]:
arr2 = pvt.loc[idx[2023, 1], :].values[:-1]/pvt.loc[idx[2023, 1], :].values[-1]
arr2

array([0.06906881, 0.19065939, 0.08321481, 0.65705704, 0.        ])

In [ ]:
arr2-arr1

array([ 0.03335074,  0.06219646, -0.02878724, -0.06675987,  0.        ])

In [ ]:
(arr2/arr1)-1

<ipython-input-230-59e3e01a186d>:1: RuntimeWarning: invalid value encountered in true_divide
  (arr2/arr1)-1


array([ 0.93372198,  0.48415881, -0.25702422, -0.09223309,         nan])